### Connect to G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Install Dependency

#### With Pre-Trained Models

In [ ]:
!pip install sentence-transformers spacy-transformers nltk laserembeddings fuzzywuzzy jellyfish python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 759.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 M

In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
!python -m spacy download en_core_web_sm
# # !python -m spacy download en_core_web_md
# # !python -m spacy download en_core_web_lg
# # !python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!git clone https://github.com/sherozshaikh/text_to_vector_embedding_pipeline.git

Cloning into 'text_to_vector_embedding_pipeline'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 78 (delta 42), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (78/78), 198.55 KiB | 6.85 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
%cd text_to_vector_embedding_pipeline

/content/text_to_vector_embedding_pipeline


In [ ]:
import time
from typing import List
import numpy as np
import pandas as pd
import os
import zipfile
import warnings
warnings.filterwarnings("ignore")
import re
from difflib import get_close_matches as difflib_get_close_matches
from difflib import SequenceMatcher as difflib_sequencematcher
from functools import partial
from fuzzywuzzy import fuzz
from fuzzywuzzy import process as fuzzywuzzy_process
from jellyfish import jaro_winkler_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Pre-Trained Models
import torch
from transformers import AutoTokenizer,AutoModel
from sentence_transformers import SentenceTransformer
from laserembeddings import Laser
import gensim.downloader as api
import spacy
from model_fetcher import HuggingFaceModelFetcher
from embedding import TextEmbedding
text_to_vector = TextEmbedding()

# For Text Processing
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords'])
# nltk_stopwords_set = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Loading Laser Embedding

In [ ]:
laser_embeddings = Laser()

##### Loading [spaCy](https://spacy.io/models) Model

In [ ]:
nlp_spacy_model = spacy.load("en_core_web_sm",enable = ["tok2vec"])

##### Loading [gensim](https://radimrehurek.com/gensim/models/word2vec.html) Model

In [ ]:
word_embedding_model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


##### Loading [Sentence Transformer](https://huggingface.co/sentence-transformers) Model

In [ ]:
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##### Loading [Pre-Trained Models](https://huggingface.co/models) Model

In [ ]:
hf_model_name:str = "google-bert/bert-base-uncased"
hf_tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
hf_model = AutoModel.from_pretrained(hf_model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### Without Pre-Trained Models

In [ ]:
!pip install fuzzywuzzy jellyfish python-Levenshtein nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 797.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.8 MB/s eta 0:00:00


In [ ]:
import time
print('Loading Libraries: ',time.ctime())
from typing import List
import numpy as np
import pandas as pd
import os
import zipfile
import warnings
warnings.filterwarnings("ignore")
import re
from difflib import get_close_matches as difflib_get_close_matches
from difflib import SequenceMatcher as difflib_sequencematcher
from functools import partial
from fuzzywuzzy import fuzz
from fuzzywuzzy import process as fuzzywuzzy_process
from jellyfish import jaro_winkler_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# For Text Processing
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords'])
# nltk_stopwords_set = set(stopwords.words('english'))

Loading Libraries:  Sun Jun 23 13:57:06 2024


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Chunk Matching Using Embeddings and Cosine Similarity

###### Objective
The script is designed to compare chunks of text extracted from two PDF files (PDF1 and PDF2) using advanced natural language processing **`(NLP)`** techniques. Each chunk represents a segment of text from the PDF, treated as a standalone document unit. The goal is to determine how closely related chunks from PDF1 are to chunks from PDF2, facilitating alignment and comparison between the two documents.

###### Methodology
1. **`Text Chunk Extraction`**: Chunks of text are extracted from PDF1 and PDF2. Each chunk can be considered as a paragraph, section, or any logical division within the document.

2. **`Embedding Generation`**: Each chunk is transformed into a numerical representation (embedding) using state-of-the-art NLP models such as Word2Vec, Pre-Trained Models like GPT or BERT, LASER Embedding, spaCy, or Sentence Transformers. These embeddings capture semantic meaning and context, enabling a more nuanced comparison than traditional methods.

3. **`Cosine Similarity Calculation`**: Using cosine similarity, the script measures how similar the embeddings of chunks from PDF1 are to those from PDF2. Cosine similarity ranges from -1 (completely dissimilar) to 1 (identical), with 0 indicating no similarity.

4. **`Mapping and Alignment`**: Based on the cosine similarity scores, chunks from PDF1 are mapped to corresponding chunks from PDF2. This mapping helps identify which chunks in PDF1 correspond most closely to chunks in PDF2, facilitating cross-document analysis and alignment.

---

###### Effectiveness
This approach is particularly effective in scenarios such as:

- **`Comparative Analysis`**: When comparing two versions of a document (e.g., before and after revisions), the script can highlight corresponding sections, aiding in understanding changes made.
  
- **`Research Synthesis`**: In academic research or literature review, matching chunks between two studies or documents can streamline comparison and synthesis of findings.

- **`Legal and Regulatory Compliance`**: For compliance reviews, mapping sections between regulatory documents ensures consistency and completeness in meeting requirements by comparing versions to identify modifications and implications.

- **`Contract Management`**: Streamlining contract revisions by comparing versions to identify changes in terms and conditions, ensuring compliance and accuracy.

- **`Financial Reporting`**:  Comparing quarterly financial statements to track discrepancies and ensure consistency in reporting across periods.

- **`Technical Documentation`**:  Aligning updates in technical manuals or engineering specifications to maintain accuracy and consistency in product documentation.

- **`Compliance Audits`**: Ensuring regulatory compliance by comparing policy updates and procedures across audits to identify gaps and ensure adherence.

---

###### Multiple Model Integration
To enhance accuracy and robustness, the script supports multiple NLP models for embedding generation:

- **`Word2Vec`**: Offers traditional word embeddings based on co-occurrence statistics, useful for capturing semantic similarities between words.

- **`Sentence Transformers`**: Transforms entire sentences or paragraphs into embeddings optimized for semantic similarity tasks, enabling comparisons of longer text segments.

- **`Pre-Trained Model (BERT/GPT)`**: Provides contextualized embeddings capturing fine-grained meaning and relationships within text.

- **`TF-IDF (Term Frequency-Inverse Document Frequency)`**: Computes the importance of each word in a document relative to a collection of documents, providing a basic measure of word relevance.

- **`Spacy Model`**: Utilizes linguistic annotations and trained pipelines to generate word vectors and document embeddings, leveraging syntactic and semantic features.

- **`LASER (Language-Agnostic SEntence Representations)`**: Produces multilingual sentence embeddings trained on large-scale parallel corpora, facilitating cross-linguistic text comparisons.

---

###### Final Weighted Score Generation
Using the script we can aggregate results from multiple models, assigning higher weights to outputs from pre-trained models like BERT, known for their superior performance in capturing semantic relationships. Lower weights are assigned to simpler models like Word2Vec, which provide complementary insights.

By synthesizing outputs from different models into a final weighted score, the script can be used to offers a comprehensive view that balances accuracy with computational efficiency, catering to diverse analytical needs.

###### Example
Consider a scenario where PDF1 contains a technical report on renewable energy projects, and PDF2 includes a funding proposal for similar projects. Using the script:
- Chunks from PDF1 describing project specifications can be mapped to corresponding sections in PDF2 detailing funding requirements.
- The cosine similarity scores indicate the degree of alignment, helping stakeholders identify critical overlaps and discrepancies between project plans and funding proposals.

This methodological approach not only enhances document comparison but also supports decision-making processes by providing structured insights into document relationships.

In [ ]:
def custom_ram_cleanup_func()->None:
  """
  Clean up global variables except for specific exclusions and system modules.

  This function deletes all global variables except those specified in
  `exclude_vars` and variables starting with underscore ('_').

  Excluded variables:
  - Modules imported into the system (except 'sys' and 'os')
  - 'sys', 'os', and 'custom_ram_cleanup_func' itself

  Returns:
  None
  """

  import sys
  all_vars = list(globals().keys())
  exclude_vars = list(sys.modules.keys())
  exclude_vars.extend(['In','Out','_','__','___','__builtin__','__builtins__','__doc__','__loader__','__name__','__package__','__spec__','_dh','_i','_i1','_ih','_ii','_iii','_oh','exit','get_ipython','quit','sys','os','custom_ram_cleanup_func',])
  for var in all_vars:
      if var not in exclude_vars and not var.startswith('_'):
          del globals()[var]
  del sys
  return None


In [ ]:
class DocMapper():
  """
  A class to find close elements links between two documents.
  """
  def __init__(self,doc1_elements_list:List[str],doc2_elements_list:List[str],doc1_elements_embedding:np.ndarray=None,doc2_elements_embedding:np.ndarray=None,threshold_:float=0.6,output_folder:str='Mapped_Attributes'):
    """
    Initialize the DocMapper class.

    Args:
    - doc1_elements_list (List[str]): List containing Doc1 Elements.
    - doc2_elements_list (List[str]): List containing Doc2 Elements.
    - doc1_elements_embedding (ndarray, optional): Doc1 Embeeding Vector.
    - doc2_elements_embedding (ndarray, optional): Doc2 Embeeding Vector.
    - threshold_ (float, optional), default = 0.6: Threshold value for filtering similarity scores.
    - output_folder (str), default = Mapped_Attributes: Output Folder Name.

    """
    self.doc1_elements_list:List[str] = doc1_elements_list
    self.doc2_elements_list:List[str] = doc2_elements_list
    self.doc1_elements_embedding:np.ndarray = doc1_elements_embedding
    self.doc2_elements_embedding:np.ndarray = doc2_elements_embedding
    self.threshold_:float = threshold_
    self.output_folder:str = self.trim_characters(stxt=output_folder).replace(' ','_')

  def __repr__(self):
    """
    Returns a string representation of the class instance.
    """
    return f"DocMapper()"

  def __str__(self):
    """
    Returns a description of the class.
    """
    return "Class to fetch Similar Doc1 Elements for given Doc2 Elements."

  def get_jaro_winkler_similarity(self,sent_1:str,sent_2:str)->float:
    """
    Calculates the Jaro-Winkler similarity between two strings.

    Args:
    - sent_1 (str): First string.
    - sent_2 (str): Second string.

    Returns:
    - float: similarity score (percentage).
    """
    return (jaro_winkler_similarity(sent_1,sent_2))*100

  def get_minhash_containment_distance(self,sent_1:str,sent_2:str)->float:
    """
    Calculates the MinHash containment distance between two strings.

    Args:
    - sent_1 (str): First string.
    - sent_2 (str): Second string.

    Returns:
    - float: MinHash containment distance score.
    """
    sent_1_len,sent_2_len,sent_1_2_len=len(sent_1),len(sent_2),len(set(sent_1).intersection(set(sent_2)))
    if sent_1_len>sent_2_len:
      return sent_1_2_len / sent_1_len
    else:
      return sent_1_2_len / sent_2_len

  def find_closest_matches_difflib(self,word_1:str,possibilities_1:List[str],n_1=10,cutoff_1=0.5)->list:
    """
    Finds closest matches using difflib's get_close_matches function.

    Args:
    - word_1 (str): Word to find matches for.
    - possibilities_1 (List[str]): List of possible matches.
    - n_1 (optional), default = 10: Maximum number of matches to return.
    - cutoff_1 (optional), default = 0.5: Similarity cutoff threshold.

    Returns:
    - list: List of closest matches.
    """
    return difflib_get_close_matches(word=word_1,possibilities=possibilities_1,n=n_1,cutoff=cutoff_1)

  def find_closest_matches_fuzzywuzzy(self,query_1:str,choices_1:List[str],limit_1=10)->list:
    """
    Finds closest matches using fuzzywuzzy's process function.

    Args:
    - query_1 (str): Query string.
    - choices_1 (List[str]): List of choices to match against.
    - limit_1 (optional): Maximum number of matches to return.

    Returns:
    - list: List of closest matches.
    """
    return fuzzywuzzy_process.extract(query=query_1,choices=choices_1,limit=limit_1)

  def process_batch_difflib(self,new_elements:str,pre_defined_elements:List[str])->dict:
    """
    Processes a batch of elements using difflib for finding closest matches.

    Args:
    - new_elements (str): New element to match.
    - pre_defined_elements (List[str]): List of pre-defined elements to match against.

    Returns:
    - dict: Dictionary containing element and its closest matches with scores.
    """
    return {'Element':new_elements,'similar':[(item,(difflib_sequencematcher(None,new_elements,item).ratio())*100) for item in self.find_closest_matches_difflib(word_1=new_elements,possibilities_1=pre_defined_elements)]}

  def process_batch_fuzzywuzzy(self,new_elements:str,pre_defined_elements:List[str])->dict:
    """
    Processes a batch of elements using fuzzywuzzy for finding closest matches.

    Args:
    - new_elements (str): New element to match.
    - pre_defined_elements (List[str]): List of pre-defined elements to match against.

    Returns:
    - dict: Dictionary containing element and its closest matches with scores.
    """
    return {'Element':new_elements,'similar':self.find_closest_matches_fuzzywuzzy(query_1=new_elements,choices_1=pre_defined_elements)}

  def trim_characters(self,stxt:str='')->str:
    """
    Removes non-alphanumeric characters from a string.

    Args:
    - stxt (str): Input string.

    Returns:
    - str: String with non-alphanumeric characters removed.
    """
    return re.compile(pattern=r'\s+').sub(repl=r' ',string=str(re.compile(pattern=r'[^a-zA-Z\d]').sub(repl=r' ',string=str(stxt)))).strip()

  def create_final_folder(self)->None:
    """
    Creates Output Folder.
    If the folder already exists, it is first removed along with all its contents, and then a new empty folder is created.

    Returns:
    - None
    """
    if os.path.exists(path=self.output_folder):
      os.rmdir(path=self.output_folder)
    os.mkdir(path=self.output_folder)
    return None

  def create_final_zip(self)->None:
    """
    Creates a ZIP archive of all the contents.
    This method walks through the directory structure, adds all files to a ZIP archive, and stores it as '.zip'.

    Returns:
    - None
    """
    # Creates ZIP
    with zipfile.ZipFile(file=self.output_folder+'.zip',mode='w',compression=zipfile.ZIP_DEFLATED) as zip_file:
      for all_root,all_dirs,all_files in os.walk(self.output_folder):
        for file_1 in all_files:
          temp_file_path = os.path.join(all_root,file_1)
          zip_file.write(
            temp_file_path,
            os.path.relpath(temp_file_path,self.output_folder)
            )

    zip_file_path:str = self.output_folder+'.zip'
    target_folder_path:str = self.output_folder
    os.rename(os.path.abspath(zip_file_path),os.path.abspath(os.path.join(target_folder_path,zip_file_path)))
    return None

  def pre_processing_text_values(self,txt:str='',is_lower:bool=True,remove_characters:bool=True)->str:
    """
    Pre-processes text values by lowercasing, removing non-alphanumeric characters, and tokenizing.

    Args:
    - txt (str): Input text.
    - is_lower (bool, optional), default = True: Convert text to lowercase.
    - remove_characters (bool, optional), default = True: Remove non-alphanumeric characters.

    Returns:
    - str: Pre-processed text.
    """
    if is_lower:
      txt:str=str(txt).lower().strip()
    else:
      txt:str=str(txt).strip()

    if remove_characters:
      txt:str=self.trim_characters(stxt=txt)
    else:
      pass

    return ' '.join([x for x in word_tokenize(txt) if x.isalnum()])

  def get_all_similarity_scores(self,row1:pd.Series)->pd.Series:
    """
    Calculates similarity scores between Doc1 and Doc2 Elements using various metrics.

    Args:
    - row1 (pd.Series): Input row containing 'doc2_elements' and 'doc1_elements'.

    Returns:
    - pd.Series: Series with similarity scores appended.
    """
    doc1_,doc2_=row1['doc1_elements'],row1['doc2_elements']
    for metric_name,metric_func in high_score_text_metrics.items():
      if metric_name in ['get_minhash_containment_distance']:
        row1[metric_name.replace('get_','high_t_')]=round(number=metric_func(sent_1=doc1_,sent_2=doc2_)*100,ndigits=6)
      else:
        row1[metric_name.replace('get_','high_t_')]=round(number=metric_func(sent_1=doc1_,sent_2=doc2_),ndigits=6)
    row1['Score']:float = round(number=(row1['Score']*100),ndigits=6)
    return row1

  def calculate_similarity_tfidf(self,texts1:List[str],texts2:List[str])->np.ndarray:
    """
    Calculates TF-IDF cosine similarity between two lists of texts.

    Args:
    - texts1 (List[str]): List of first texts.
    - texts2 (List[str]): List of second texts.

    Returns:
    - np.ndarray: Similarity score matrix.
    """
    tfidf_vectorizer = TfidfVectorizer(decode_error = 'strict',use_idf = True,smooth_idf = True,binary = False,lowercase = True,max_features = 30_000,dtype = np.float32,ngram_range = (1,4),stop_words = 'english').fit(texts1+texts2)
    texts1_matrix:np.ndarray = tfidf_vectorizer.transform(texts1)
    texts2_matrix:np.ndarray = tfidf_vectorizer.transform(texts2)
    similarity_score_matrix:np.ndarray = cosine_similarity(texts1_matrix,texts2_matrix)
    return similarity_score_matrix

  def calculate_similarity_score(self,texts1_matrix:np.ndarray,texts2_matrix:np.ndarray)->np.ndarray:
    """
    Calculates cosine similarity between two matrices of texts.

    Args:
    - texts1_matrix (np.ndarray): First matrix of texts.
    - texts2_matrix (np.ndarray): Second matrix of texts.

    Returns:
    - np.ndarray: Similarity score matrix.
    """
    return cosine_similarity(texts1_matrix,texts2_matrix)

  def filter_similarity_matrix(self,similarity_matrix:np.ndarray,threshold_val:float=0.65)->pd.DataFrame:
    """
    Filters similarity matrix based on a threshold value.

    Args:
    - similarity_matrix (np.ndarray): Similarity score matrix.
    - threshold_val (float, optional), default = 0.65: Threshold value for similarity. Update using "threshold_" during initialization

    Returns:
    - pd.DataFrame: Filtered DataFrame with relevant attributes and scores.
    """
    relevant_indices:np.ndarray = np.argwhere(similarity_matrix > threshold_val)
    ids_1:np.ndarray = relevant_indices[:, 0]
    ids_2:np.ndarray = relevant_indices[:, 1]
    filtered_scores:np.ndarray = similarity_matrix[ids_1,ids_2]
    results:list = [{
        'doc1_elements': self.doc1_elements_list[i],
        'doc2_elements': self.doc2_elements_list[j],
        'Score': k,
    } for i, j, k in zip(ids_1, ids_2, filtered_scores)]
    results_df:pd.DataFrame = pd.DataFrame(results).sort_values(by=['doc1_elements','Score'],ascending=[True,False])
    return results_df

  def main(self)->None:
    """
    Main function to perform attribute mapping and write results to a CSV file.
    """
    # Define similarity metrics
    high_score_text_metrics:dict={
      'get_jaro_winkler_similarity':self.get_jaro_winkler_similarity,
      'get_minhash_containment_distance':self.get_minhash_containment_distance,
      }

    # Calculate similarity scores based on the availability of an embedding model
    if (self.doc1_elements_embedding is not None and np.any(self.doc1_elements_embedding != None)) and (self.doc2_elements_embedding is not None and np.any(self.doc2_elements_embedding != None)):
      similarity_score:np.ndarray = self.calculate_similarity_score(texts1_matrix=self.doc1_elements_embedding,texts2_matrix=self.doc2_elements_embedding)
    else:
      processed_doc1_elements_list:list=[self.pre_processing_text_values(txt=x,is_lower=True,remove_characters=True) for x in self.doc1_elements_list]
      processed_doc2_elements_list:list=[self.pre_processing_text_values(txt=x,is_lower=True,remove_characters=True) for x in self.doc2_elements_list]
      similarity_score:np.ndarray = self.calculate_similarity_tfidf(texts1=processed_doc1_elements_list,texts2=processed_doc2_elements_list)

    # Filter and process the similarity matrix
    mapped_result_df:pd.DataFrame = self.filter_similarity_matrix(similarity_matrix=similarity_score,threshold_val=self.threshold_)
    temp_df:pd.DataFrame = mapped_result_df['doc1_elements'].value_counts().reset_index()
    temp_df:pd.DataFrame = temp_df[temp_df['count']>1]
    filtered_df:pd.DataFrame = mapped_result_df[mapped_result_df['doc1_elements'].isin(temp_df['doc1_elements'].tolist())]
    del temp_df

    # Group by Doc2 Elements and find nearest matches using difflib and fuzzywuzzy
    grouped_df:pd.DataFrame = filtered_df[['doc1_elements','doc2_elements']].groupby(by=['doc1_elements']).agg(list).reset_index()
    nearest_matches_difflib:pd.Series = grouped_df.apply(lambda x: self.process_batch_difflib(new_elements=x['doc1_elements'],pre_defined_elements=x['doc2_elements']),axis=1)
    nearest_matches_fuzzywuzzy:pd.Series = grouped_df.apply(lambda x: self.process_batch_fuzzywuzzy(new_elements=x['doc1_elements'],pre_defined_elements=x['doc2_elements']),axis=1)

    if not nearest_matches_difflib.empty:
      nearest_matches_difflib_df:pd.DataFrame = pd.DataFrame([(entry['Element'],similar[0],similar[1]) for entry in nearest_matches_difflib for similar in entry['similar']],columns=['doc1_elements','doc2_elements','difflib'])
    else:
      nearest_matches_difflib_df:pd.DataFrame = pd.DataFrame(columns=['doc1_elements','doc2_elements','difflib'])

    if not nearest_matches_fuzzywuzzy.empty:
      nearest_matches_fuzzywuzzy_df:pd.DataFrame = pd.DataFrame([(entry['Element'],similar[0],similar[1]) for entry in nearest_matches_fuzzywuzzy for similar in entry['similar']],columns=['doc1_elements','doc2_elements','fuzzywuzzy'])
    else:
      nearest_matches_fuzzywuzzy_df:pd.DataFrame = pd.DataFrame(columns=['doc1_elements','doc2_elements','fuzzywuzzy'])

    # Post-Processing
    nearest_matches_df:pd.DataFrame = pd.merge(left=nearest_matches_difflib_df,right=nearest_matches_fuzzywuzzy_df,on=['doc1_elements','doc2_elements'],how='outer')
    nearest_matches_df:pd.DataFrame = nearest_matches_df[(nearest_matches_df['difflib']>self.threshold_) & (nearest_matches_df['fuzzywuzzy']>self.threshold_)]
    nearest_matches_df:pd.DataFrame = nearest_matches_df[['doc1_elements','doc2_elements','difflib']]
    nearest_matches_df.columns = ['doc1_elements','doc2_elements','Score']
    mapped_result_df['Score']:pd.Series = mapped_result_df['Score']*100
    mapped_result_df['Score']:pd.Series = mapped_result_df['Score'].round(4)
    consolidated_result_df:pd.DataFrame = pd.concat(objs=[mapped_result_df,nearest_matches_df])
    consolidated_result_df:pd.DataFrame = consolidated_result_df.sort_values(by=['doc1_elements','Score'],ascending=[True,False])
    consolidated_result_df:pd.DataFrame = consolidated_result_df.drop_duplicates(subset=['doc1_elements','doc2_elements'])
    doc1_not_found_df:pd.DataFrame = pd.DataFrame(data=set(self.doc1_elements_list).difference(set(consolidated_result_df['doc1_elements'].drop_duplicates().dropna().tolist())),columns=['doc1_elements'])
    doc2_not_found_df:pd.DataFrame = pd.DataFrame(data=set(self.doc2_elements_list).difference(set(consolidated_result_df['doc2_elements'].drop_duplicates().dropna().tolist())),columns=['doc2_elements'])
    consolidated_result_df.columns = ['Doc1_elements','Doc2_elements','Score']
    del nearest_matches_difflib_df,nearest_matches_fuzzywuzzy_df,grouped_df,nearest_matches_difflib,nearest_matches_fuzzywuzzy,mapped_result_df,nearest_matches_df

    # Finalize and write the results to a CSV file
    self.create_final_folder() # create folder

    consolidated_result_df.to_hdf(path_or_buf=self.output_folder+'/Mapping.h5',key='compress',mode='w',encoding='utf-8') # save in hdf file format

    with pd.ExcelWriter(path=self.output_folder+'/Mapping.xlsx',mode='w') as file_writer: # save in excel file format
      consolidated_result_df.to_excel(excel_writer=file_writer,sheet_name='Mapping',index=False)
      doc1_not_found_df.to_excel(excel_writer=file_writer,sheet_name='Doc1_NotFound',index=False)
      doc2_not_found_df.to_excel(excel_writer=file_writer,sheet_name='Doc2_NotFound',index=False)
    del consolidated_result_df,doc1_not_found_df,doc2_not_found_df

    self.create_final_zip() # create zip
    print('wrote the final output to local: ',time.ctime())
    return None


### Simulation Run

In [ ]:
doc1_elements:list=[
  "Series",
  "Type",
  "Material",
  "Color",
  "Finish",
  "Insulation Type",
  "Frame",
  "Suitable For Use With",
  "Indicator Type",
  "Number of Outlets",
  "Operating Cycles",
  "Response Time",
  "Reset Type",
  "Insulation",
  "Timing",
  "Full Load Current",
  "Leakage Current",
  "Motor Power",
  "Overload Relay",
  "Setting",
  "Starting Modes",
  "Class",
  "Electrical Ratings",
  "Nominal Discharge Current",
  "Clamping Voltage",
  "Energy Rating",
  "Modes of Protection",
  "Peak Surge Current",
  "Efficiency",
  "Power Factor",
  "Duty Cycle",
  "Primary Voltage",
  "Secondary Voltage",
  "Specifications",
  "Creepage Distance",
  "Contact Configuration",
  "Winding Material",
  "Control",
  "Communication",
  "Plug Type",
  "Meter Interconnect",
  "Light Color",
  "Lighting Technology",
  "Housing Finish",
  "Receptacle",
  "Cable Size",
  "Configuration",
  "Fuse Amperage Rating",
  "Number of Receptacles",
  "Shade Type",
  "Contact Form",
  "Enclosure",
  "Receptacle Type",
  "Trip Amperage",
  "Cord Diameter",
  "Mounting",
  "Number of Inputs",
  "Number of Outputs",
  "Bezel/Housing",
  "IR Rating",
  "Handle Color",
  "Handle Mounting Position",
  "Handle/Operation",
  "Handle to Device Connection",
  "Shaft Length",
  "Accuracy",
  "Number of Ports",
  "Contact Mode",
  "Circuit Breaker Type",
  "Number of Feeders",
  "Input Type",
  "Supply Voltage",
  "Rating of Feeders",
  "Sensing Method",
  "Interface",
  "Hole Spacing",
  "Number of Contacts",
  "Bolt/Stud Size",
  "Conductor Range",
  "Hex Size",
  "Fuse Type",
  "Contact Type",
  "Maximum Load Current",
  "Break Time",
  "Expansion Slot",
  "Number of Auxiliary Contacts",
  "Resolution",
  "Body Type",
  "Element",
  "Body Size",
  "Legend",
  "Throw",
  "Capacity",
  "Start Type",
  "Cover Type",
  "Cord Length",
  "Indicating/Non-Indicating",
  "Rejection/Non-Rejection",
  "Operation Mode",
  "Light Source",
  "Sensing Range",
  "Vibration",
  "Cable Entry Size",
  "Cut-Off Size",
  "Frequency Response",
  "Measuring Range",
  "Sensing Beam",
  "Sensing Distance",
  "Sensing Position",
  "Switching Speed",
  "Cable Type",
  "Fuse Size",
  "Lock",
  "Wire Type",
  "Number of Outlet Rows",
  "Number of Power Switches",
  "Connection",
  "Panel Size",
  "Legend Color",
  "Bus Rating",
  "Power Source",
  "Input Voltage",
  "Number of Heads",
  "Output Voltage",
  "Counts per Second",
  "Input",
  "Input Signal",
  "Number of Digits",
  "On/Off Cycles",
  "Output",
  "Cable Openings",
  "Operating Mode",
  "Trade Size",
  "Coverage",
  "Memory",
  "Number of Conductors",
  "Number of Gangs",
  "Cable Length",
  "USB Quantity",
  "USB Rating",
  "USB Type",
  "Wiring Method",
  "Contacts",
  "Indicator",
  "Interrupt Media",
  "Operating Media",
  "Type of Release",
  "Sensor Type",
  "Control Output",
  "Response Frequency",
  "Time Delay",
  "Viewing Angle",
  "Operational Mode",
  "Programming",
  "Primary Connections/Pole",
  "Secondary Connection/Pole",
  "Switching Positions",
  "Connector Size",
  "Noise Attenuation",
  "Power Efficiency",
  "Signal Range",
  "Cable Material",
  "Connector Material",
  "Wire Pin Colors",
  "Communication Ports",
  "Contact Material",
  "Duration",
  "Product Type",
  "Services Covered",
  "Designation",
  "Disc Type",
  "Diskette Size",
  "Fixture Type",
  "License",
  "Maintenance Type",
  "Number of Terminals",
  "Operating System Compatibility",
  "Platform",
  "Portability",
  "Security",
  "Title",
  "Sensitivity",
  "Wiring Configuration",
  "Input/Output Capacity",
  "Number of Circuits",
  "Style",
  "Bus/Backplane Load",
  "Hub Size",
  "Lens Color",
  "Lens Material",
  "Maximum I/O Capacity",
  "Number of Functions",
  "Program Scan Time",
  "Dielectric Strength",
  "Enclosure Box Size",
  "Bit Rate",
  "Data Transfer Rate",
  "Diaphragm Material",
  "Differential",
  "License Type",
  "Maximum Contact Block",
  "Maximum Pressure",
  "Power Output",
  "Protocol",
  "Starter Size",
  "Cooling",
  "Function",
  "Hub Type",
  "Input Fuse",
  "Knockout Size",
  "Mounting Orientation",
  "NEMA Plug Configuration",
  "NEMA Receptacle Configuration",
  "Number of Jaws",
  "Outlet Type",
  "Conversion Rate",
  "Fuse Diameter",
  "Number of Slots",
  "Slot Size",
  "Slot Spacing",
  "Step Response",
  "Closed Contact Resistance",
  "Current Ratio",
  "Fuse Holder Code",
  "Fuse Length",
  "Primary Current",
  "Scan Time",
  "Secondary Current",
  "Transfer Rate",
  "Wire Clamp Type",
  "BIL Rating",
  "Duty Cycle Rating",
  "Engine Size",
  "Fuel Tank Capacity",
  "Fuel Type",
  "Insulator Material",
  "Leakage to Ground",
  "Operating Voltage",
  "Starter Type",
  "Terminal Wire Range",
  "Loading Depth",
  "Power Supply",
  "Rated Breaking Capacity",
  "Rung Spacing",
  "Side Rail Height",
  "Specifications Met",
  "Timing Response",
  "Grip",
  "Mounting Hole Size",
  "Case Material",
  "Construction",
  "Cutout",
  "Orientation",
  "Auxiliary Contacts",
  "Barrier Style",
  "Center Spacing",
  "Field Color",
  "Frame Type",
  "Legend Size",
  "Load Rating",
  "Plate Shape",
  "Screw Size",
  "Bolt Circle Diameter",
  "Bolt Size",
  "Brightness",
  "Charging Time",
  "Display Resolution",
  "Display Size",
  "Electrode",
  "Electrolyte",
  "Internal Impedance",
  "Maximum Discharge Current",
  "Mounting Height",
  "Number of Keys",
  "Number of Operators",
  "Projection",
  "Shelf Life",
  "Software",
  "Sound Level",
  "Tip Type",
  "Viewing Area",
  "Flash Rate",
  "Flash Type",
  "Frame Base Material",
  "Frame Base Size",
  "Frequency Range",
  "Insert Frame",
  "Maximum Breaker Current Rating",
  "Number of Lights",
  "Plug Current Rating",
  "Charging Current",
  "Contact Resistance",
  "Filament",
  "Labels per Roll",
  "Lamp Finish",
  "Block Material",
  "Breaking Strength",
  "Chemical Composition",
  "Contactor Size",
  "Container Size",
  "Container Type",
  "Curing Time",
  "Domestic/Import",
  "Form",
  "Gland Size",
  "Imprint Area",
  "Imprinting",
  "Incandescent Equivalent",
  "Insulation Size",
  "Load Regulation",
  "Mesh Length",
  "Odor/Scent",
  "pH Range",
  "Protection",
  "Sensor",
  "Shield Size",
  "Speed Range",
  "Viscosity",
  "VOC Content",
  "Voltage Input Range",
  "Blade End Style",
  "Charge Current",
  "Charge Voltage",
  "Chemical System",
  "Communication Output",
  "Communication Speed",
  "Compatibility",
  "Coupling Nut Material",
  "Die Code",
  "Discharge Cut-Off Voltage",
  "Disconnect Current Rating",
  "Disconnect Type",
  "Displayed Parameters",
  "Electrical Connections",
  "Handle Material",
  "Handle Size",
  "Hub Style",
  "Humidity",
  "Insert Material",
  "Life Hour",
  "Nominal Voltage",
  "Number of Holes",
  "On/Off Ratio",
  "Operating Force",
  "O-Ring Material",
  "Outlet",
  "Overload Current",
  "Overload Type",
  "Overmold Material",
  "Ports",
  "Pulse Duration",
  "Resistance",
  "Service Feed",
  "Shell Material",
  "Slot End Style",
  "Time Setting",
  "Travel",
  "Tray Depth",
  "Type of Indicators",
  "Actuator Size",
  "Ballast Load",
  "Bare Tool/Kit",
  "Bar Thickness",
  "Bus Current",
  "Bus Material",
  "Button Type",
  "Cable/Wire Size",
  "Capacitance",
  "Communication Rate",
  "Conductor Material",
  "Conduit Size",
  "Connection Size",
  "Construction Style",
  "Cord Size",
  "Crimping Capacity",
  "Crimping Force",
  "Diameter - Thread Size",
  "Discharge",
  "End Angle",
  "End Type",
  "Eye Diameter",
  "Eye Length",
  "Eye Outer Size",
  "Eye Type",
  "Ferrule Hole Size",
  "Ferrule Hole Spacing",
  "Ferrule Length",
  "Flange Size",
  "For Use With",
  "Fuse Clip Amperage",
  "Fuse Provision",
  "Fuse Voltage Rating",
  "Grille Material",
  "Grille Shape",
  "Grille Size",
  "Hardness",
  "Hole Size",
  "Hole Style",
  "Inner Length",
  "Inner Width",
  "Insulated",
  "Insulated Handle",
  "Jaw Material",
  "Jaw Type",
  "Lamp Designation",
  "Logging Memory",
  "Measurement System",
  "Mesh Type",
  "Motor Type",
  "Non-Sparking",
  "Number of Lugs",
  "Number of Pilot Lights",
  "Number of Shelves",
  "Nut Material",
  "Opening Size",
  "Overall Depth",
  "Pin Diameter",
  "Pin Material",
  "Pin Type",
  "Power Consumption",
  "Power Dissipation",
  "Replaceable Handle",
  "Response Delay",
  "Rod Size",
  "Shearing Capacity",
  "Shelf Capacity",
  "Shelf Size",
  "Stud Length",
  "Terminal Base Unit",
  "Terminals/Conductor Size",
  "Thread Direction",
  "Timing Range",
  "Tungsten Rating",
  "Width Across Flats",
  "Wiring Type",
  "Wrench Size",
  "Yield Strength",
  "Air Pressure",
  "Arm Length",
  "Arm Width",
  "Band Diameter",
  "Band Length",
  "Bar Length",
  "Bar Width",
  "Barrel Diameter",
  "Barrel Length",
  "Barrel Size",
  "Base Diameter",
  "Base Height",
  "Base Length",
  "Base Size",
  "Base Width",
  "Bit Length",
  "Bit Size",
  "Blade Diameter",
  "Blade Height",
  "Blade Length",
  "Blade Pitch",
  "Blade Range",
  "Blade Size",
  "Drill Bit Size",
  "Maximum Drill Bit Size",
  "Maximum Solid Bit Capacity",
  "Minimum Drill Bit Size",
  "Tool Bit Size",
  "Shank Size",
  "Operator Type",
  "Bore Diameter",
  "Terminal Type",
  "Cabling Type",
  "Operation Type",
  "Shrink Ratio",
  "Blade Thickness",
  "Keyway",
  "Lens Type",
  "Magnet Type",
  "Printer Type",
  "Number of Poles",
  "Amperage Rating",
  "Control Type",
  "Cutting Depth",
  "Drill Size - Decimal Inch",
  "End Style",
  "Head Type",
  "Lamp Type",
  "Nominal Size",
  "Shape",
  "Size",
  "Tray Width",
  "Flute Length",
  "Display Type",
  "Adhesive Material",
  "Backing Material",
  "Blade Material",
  "Bushing Type",
  "Color Temperature",
  "Fitting/Connector Type",
  "Heating Capacity",
  "Insert Style",
  "Insulation Material",
  "Jacket Color",
  "Jacket Material",
  "Light Output",
  "Lubrication Type",
  "Lumens",
  "Material Grade",
  "Motor RPM",
  "Number of Teeth",
  "Point Type",
  "Rake",
  "Softgoods Material",
  "Teeth per Inch",
  "Tensile Strength",
  "Trim Material",
  "Lamp Wattage",
  "Gear Ratio",
  "Hole Diameter",
  "Nominal Capacity",
  "Refrigerant Type",
  "Thread Size",
  "Bottom Type",
  "Flute Type",
  "Frequency Rating",
  "Housing Material",
  "Mounting Type",
  "Outside Diameter",
  "Phase",
  "Port Type",
  "Thread Type",
  "ANSI Code",
  "Body Material",
  "Color Rendering Index (CRI)",
  "Connection Type",
  "Drive Type",
  "Grade",
  "Insulation Class",
  "Load Capacity",
  "Operating Temperature",
  "Temperature Rating",
  "Torque Rating",
  "Angle",
  "Bend Radius",
  "Flow Rate",
  "Horsepower",
  "Interrupt Rating",
  "Motor HP",
  "Pitch",
  "Shaft Size",
  "Thickness",
  "Number of Flutes",
  "Average Life",
  "Connector Type",
  "Cutting Direction",
  "Enclosure Type",
  "Lamp Base Type",
  "Output Type",
  "Pressure Rating",
  "Shank Diameter",
  "Shank Type",
  "Voltage Rating",
  "Chipbreaker",
  "Coil Voltage",
  "Conductor Size",
  "Depth",
  "Diameter",
  "Fixture Wattage",
  "Height",
  "Input Maximum Speed",
  "Inscribed Circle",
  "Keyway Size",
  "Length",
  "Length of Cut",
  "Maximum Speed",
  "NEMA Rating",
  "Outlet Size",
  "Output Maximum Speed",
  "Overall Height",
  "Overall Length",
  "Overall Width",
  "Speed",
  "Temperature Range",
  "Wattage",
  "Width",
  "Inside Diameter",
  "Actuator",
  "Actuator Type",
  "Blade Width",
  "Closure Type",
  "Conductor Type",
  "Cord Type",
  "Corner Radius",
  "Cutting Edge Material",
  "Drill Point Angle",
  "Handle Type",
  "Helix Angle",
  "IEC Rating",
  "Insert Size",
  "Jaw Size",
  "Number of Doors",
  "Number of Lamps",
  "Number of Wires",
  "Operator",
  "Point Angle",
  "Switch Function",
  "Trip Type",
  "Wire Size",
  "Bearing Type",
  "Environmental Conditions",
  "Media",
  "Pitch Diameter",
  "45 DEG Capacity",
  "90 DEG Capacity",
  "Abrading Application",
  "Abrasive Depth",
  "Abrasive Length",
  "Abrasive Material",
  "Abrasive Width",
  "Absorbs",
  "Absorption Capacity",
  "AC Cable Size",
  "AC Current Accuracy",
  "AC Response",
  "Acceptance Capacity",
  "Acceptance Factor",
  "Acceptance Volume",
  "Access Type",
  "Accessory Type",
  "Action",
  "Activates at Temperature",
  "Activates By",
  "Activation Type",
  "Active Circuits",
  "Actual Diameter",
  "Actuating Force",
  "Actuation Type",
  "Actuator Length",
  "Actuator Location",
  "Actuator Material",
  "ADA Compliance (Yes/No)",
  "Adapter Model Number Compatibility",
  "Adapter Shank Type",
  "Adapter Size",
  "Adapter Type",
  "Adapter/Shank Diameter",
  "Additives",
  "Adheres to",
  "Adhesion Strength",
  "Adhesive Backed",
  "Adhesive Presence",
  "Adhesive Type",
  "Adjustable Die (Yes/No)",
  "Adjustable Increments",
  "Adjustable Range",
  "Adjustable Tension",
  "Adjustment Range",
  "Adjustment Type",
  "AFUE Rating",
  "Air Connection",
  "Air Consumption",
  "Air Diffusion Pattern",
  "Air Flow",
  "Air Flow Efficiency",
  "Air Flow Rate",
  "Air Flow Type",
  "Air Gap",
  "Air Inlet",
  "Air Inlet Gender",
  "Air Inlet Pressure",
  "Air Input Range",
  "Air Requirement",
  "Air Supply",
  "Air Supply Pressure",
  "Air Tube Diameter",
  "Air Tube Length",
  "Air Velocity",
  "Air Velocity Accuracy",
  "Air Watts",
  "Alarm Duration",
  "Alarm Setting",
  "Alarm Type",
  "Alert Mode",
  "Altitude",
  "AM Suppression",
  "Ambient Air Temperature Rating",
  "Ambient Light",
  "Ambient Light Filter",
  "Ambient Temperature Rating",
  "Amplitude Error",
  "Analog Output",
  "Anchor Diameter",
  "Anchor Material",
  "Anchor Rod Length",
  "Anchor Rod Size",
  "Anchor Size",
  "Anchorage Connection Type",
  "Anchorage Hook Gate Opening",
  "Anchorage Point Rating",
  "Angle of View",
  "Angle Radius",
  "Angular or Straight Mount Bars",
  "Ankle Style",
  "ANSI Class",
  "ANSI Cut-Resistance Level",
  "ANSI Electrical Class Rating",
  "ANSI Heat Level",
  "ANSI Impact Rating",
  "ANSI Puncture-Resistance Level",
  "ANSI/ISO Designation",
  "Answer Mode",
  "Antenna Length",
  "Antenna Size",
  "Antenna Type",
  "Anvil Key Compatibility",
  "Anvil Orientation",
  "Anvil Screw Compatibility",
  "Anvil Type",
  "Aperture Angle",
  "Aperture Area",
  "Aperture Ratio",
  "Aperture Size",
  "Apparatus Mounting Type",
  "Apparent Power",
  "Appearance",
  "Applicable Materials",
  "Application Method",
  "Application Rating",
  "Application Speed",
  "Application Time",
  "Application Type",
  "Applied Length",
  "Approximate Weight",
  "Arbitrary Length",
  "Arbor Hole Diameter",
  "Arbor Hole Size",
  "Arbor Length",
  "Arbor Number",
  "Arbor Number Compatibility",
  "Arbor Size",
  "Arbor/Shank Size",
  "Arc Rating",
  "Arc Resistance",
  "Area Expanded",
  "Area of Concentration",
  "Area Protected",
  "Arm Height",
  "Arm Material",
  "Arm Size",
  "Arm Span",
  "Arm Type",
  "Armature Depth",
  "Armature Length",
  "Armature Width",
  "Arms",
  "Arrester",
  "ASME Yes/No",
  "Assembly Clearance",
  "ASTM Class",
  "ATPV Rating",
  "Attachment Type",
  "Attenuation",
  "Attenuation Rating",
  "Attenuator Type",
  "Audible Alert",
  "Audible Indicator",
  "Audio Alarm",
  "Audio Formats",
  "Audio I/O Type",
  "Audio Outputs",
  "Audio Specifications",
  "Audio Type",
  "Auger Diameter",
  "Authentication Type",
  "Author",
  "Auto Leveling Range",
  "Auto Power Off",
  "Auto Ranging",
  "Automatic/Non Automatic",
  "Auxiliary Contact Rating",
  "Auxiliary Input",
  "Average Brush Life",
  "Average On-State Current",
  "Avery Template Number",
  "Axial Rake Angle",
  "Axle Type",
  "Back of Bar to Cutting Edge",
  "Back of Head Diameter",
  "Back Panel Size",
  "Back Pressure",
  "Back Type",
  "Backflex Radius",
  "Backflow Type",
  "Background Color",
  "Backing",
  "Backing Texture",
  "Backing Weight",
  "Backlash",
  "Backlight Type",
  "Backset",
  "Backup Time",
  "Backward Compatibility",
  "Bacteria Removal Rate",
  "Bail Diameter",
  "Bail Opening Height",
  "Bail Opening Width",
  "Bail Size",
  "Bail Type",
  "Ball Diameter",
  "Ball Material",
  "Ball Stem Diameter",
  "Ball Valve/Gate Valve Type",
  "Ball/Spool Size",
  "Ballast Factor",
  "Ballast Grade",
  "Ballast Mounting",
  "Ballast Test",
  "Ballast Type",
  "Balls per Foot",
  "Balls per Outer Race",
  "Band Material",
  "Band Position",
  "Band Type",
  "Bandwidth",
  "Bar Depth",
  "Bar Diameter",
  "Bar End Angle",
  "Bar Height",
  "Bar Material",
  "Bar Opening",
  "Bar Size",
  "Bar Spacing",
  "Bar Type",
  "Bar/Chain Length",
  "Bar/Chain Material",
  "Bar/Chain Oiler",
  "Bar/Chain Size",
  "Barb Length",
  "Barrel Color",
  "Barrel Height",
  "Barrel Material",
  "Barrel Outer Diameter",
  "Barrel Type",
  "Base Color",
  "Base Depth",
  "Base Dimensions",
  "Base Material",
  "Base Material Thickness Range",
  "Base Pin",
  "Base Plate Size",
  "Base Radius Range",
  "Base Shape",
  "Base Spread",
  "Base Style",
  "Base Thickness",
  "Base to Bore Centerline",
  "Base to Center Height of Shaft",
  "Base Type",
  "Basic Impulse Level (BIL)",
  "Basin Capacity",
  "Basin Color",
  "Basin Dimensions",
  "Basin Height",
  "Basin Material",
  "Basin Type",
  "Basket Hitch Capacity",
  "Basket Material",
  "Bath Therapy Type",
  "Bathing Well Size",
  "Battery",
  "Battery Capacity",
  "Battery Chemistry",
  "Battery Included",
  "Battery Life",
  "Battery Recharge Time",
  "Battery Size",
  "Battery Type",
  "Battery Voltage",
  "Bead Diameter",
  "Beam Angle",
  "Beam Color",
  "Beam Depth",
  "Beam Distance",
  "Beam Length",
  "Beam Pattern",
  "Beam Spread",
  "Beam Thickness",
  "Beam Type",
  "Beam Width",
  "Bearing Bore",
  "Bearing Diameter",
  "Bearing ID Range",
  "Bearing Material",
  "Bearing Maximum Outside Diameter",
  "Bearing Maximum Weight",
  "Bearing Width",
  "Bearing/Bushing Type",
  "Bearings",
  "Bed Length",
  "Bed Width",
  "Begins to Harden",
  "Bell Diameter",
  "Belt Color",
  "Belt Length",
  "Belt Location",
  "Belt Material",
  "Belt Message",
  "Belt Orientation",
  "Belt Pulley Diameter",
  "Belt Size",
  "Belt Speed",
  "Belt Strength",
  "Belt Table Size",
  "Belt Thickness",
  "Belt Type",
  "Belt Width",
  "Bench Size",
  "Bend Angle",
  "Bend Type",
  "Bender Type",
  "Bending Angle",
  "Bending Capacity",
  "Bending Radius",
  "Bending Radius Capillary Tube",
  "Between Frame Width",
  "Bevel Angle",
  "Bevel Capacity",
  "Bevel Stops",
  "Bin Color",
  "Bin Depth",
  "Bin Height",
  "Bin Size",
  "Bin Storage Capacity",
  "Bin Type",
  "Bin Width",
  "Binding Type",
  "Bit Material",
  "Bit Retainer Type",
  "Bit Type",
  "Blade Action",
  "Blade Angle",
  "Blade Change Method",
  "Blade Cutting Width",
  "Blade Design",
  "Blade Finish",
  "Blade Flexibility",
  "Blade Life",
  "Blade Orientation",
  "Blade Point Style",
  "Blade Radius",
  "Blade Rotation Positions",
  "Blade Shape",
  "Blade Side",
  "Blade Spacing",
  "Blade Speed",
  "Blade Style",
  "Blade Tilt",
  "Blade Tip Material",
  "Blade to Ceiling Height",
  "Blade Trade Number",
  "Blade Type",
  "Blade Width/Diameter",
  "Bleed Rate",
  "Block Size",
  "Block Type",
  "Blow-Off Size",
  "Blower Speed",
  "Blower Wheel Diameter",
  "Blows per Minute",
  "Board Color",
  "Board Finish",
  "Board Material",
  "Board Thickness",
  "Body Color",
  "Body Diameter",
  "Body Diameter Workpiece Side",
  "Body Dimensions",
  "Body Height",
  "Body Length",
  "Body Shank Diameter",
  "Body Shell Material",
  "Body Thickness",
  "Body Width",
  "Boiler Output",
  "Boiling Point",
  "Bolt Capacity",
  "Bolt Center-to-Center Length",
  "Bolt Center-to-Center Width",
  "Bolt Clearance Depth",
  "Bolt Diameter",
  "Bolt Hole Diameter",
  "Bolt Hole Size",
  "Bolt Length",
  "Bolt Material",
  "Bolt Range",
  "Bolt to Seat Front",
  "Bolt Type",
  "Bolt/Rod Size",
  "Bolted Circuits",
  "Bond Type",
  "Bonds To",
  "Bonnet Material",
  "Book Format",
  "Book Title",
  "Boost Rating",
  "Boot Type",
  "Border Type",
  "Bore Depth",
  "Bore Length",
  "Bore Size",
  "Bore Type",
  "Boring Bar Hole Diameter",
  "Boring Bar Type",
  "Boring Head Compatibility",
  "Bottom Depth",
  "Bottom Diameter",
  "Bottom Return Size",
  "Bottom Terminal",
  "Bottom Width",
  "Bow and Twist",
  "Bowl Configuration",
  "Bowl Depth",
  "Bowl Diameter",
  "Bowl Dimensions",
  "Bowl Length",
  "Bowl Material",
  "Bowl Shape",
  "Bowl Size",
  "Bowl Width",
  "Box End Thickness",
  "Box End Width",
  "Box Entry",
  "Box Length",
  "Box Quantity",
  "Box Size",
  "Box Type",
  "Bracket Mounting Hardware",
  "Bracket Mounting Type",
  "Brake Type",
  "Branch Lug Amps",
  "Breakdown Voltage",
  "Breaker Frame Type",
  "Breaker Identification",
  "Breaker Mounting",
  "Breaker Type",
  "Breaking Load",
  "Bridge Size",
  "Bristle Firmness",
  "Bristle Material",
  "Broach Style",
  "Brush Color",
  "Brush Diameter",
  "Brush Length",
  "Brush Material",
  "Brush Size",
  "Brush Speed",
  "Brush Type",
  "Brushless Motor",
  "BTU Rating",
  "Bubble Size",
  "Buckle Type",
  "Buff Diameter",
  "Buff Length",
  "Buff Type",
  "Bulb Base",
  "Bulb Shape",
  "Bulb Size",
  "Bulb Type",
  "Bulk Capacity",
  "Bullet Diameter",
  "Bumper Color",
  "Bundle Diameter",
  "Burner Capacity",
  "Burner Fuel",
  "Burner Power",
  "Burner/Element Style",
  "Burst Pressure",
  "Bury Depth",
  "Bus Amps",
  "Bus Height",
  "Bus Length",
  "Bus Size",
  "Bus Termination",
  "Bus Thickness",
  "Bus Type",
  "Bus Width",
  "Busbar Length",
  "Busbar Size",
  "Busbar Thickness",
  "Busbar Type",
  "Busbar Width",
  "Bushing Diameter",
  "Bushing Hole Diameter",
  "Bushing Industry Code",
  "Butt Depth",
  "Button",
  "Cabinet Color",
  "Cabinet Dimensions",
  "Cabinet Material",
  "Cabinet Shelf Capacity",
  "Cabinet Shelf Size",
  "Cabinet Type",
  "Cable Bushing",
  "Cable Capacity",
  "Cable Compatibility",
  "Cable Construction",
  "Cable Crimp Die",
  "Cable Diameter",
  "Cable Groove Radius",
  "Cable Main",
  "Cable Outer Diameter",
  "Cable Pull Strength",
  "Cable Sheath Material",
  "Cable Tap",
  "Cable Tie Capacity",
  "Cable Tie Material",
  "Cable Tie Tensile Strength",
  "Cable Tie Type",
  "Cable Tie Width",
  "Cable Well Depth",
  "Cable Width",
  "Cableway Size",
  "Cache Memory",
  "Cage Material",
  "Caliber",
  "Calibration",
  "Calibration Accuracy",
  "Calibration Method",
  "Cam Code",
  "Camera Diameter",
  "Camera Inputs",
  "Camera Resolution",
  "Camera Size",
  "Camera Type",
  "Camera Voltage",
  "Can Capacity",
  "Canopy Type",
  "Cap Diameter",
  "Cap Height",
  "Cap Material",
  "Cap Shape",
  "Cap Size",
  "Cap Width",
  "Capacitance Accuracy",
  "Capacitance Range",
  "Capacitor Rating",
  "Capacity at Maximum Height",
  "Capacity at Minimum Height",
  "Capacity of Mild Steel",
  "Capacity per Shelf",
  "Capacity Range",
  "Capillary Length",
  "Capillary Type",
  "Carbon Dioxide Concentration",
  "Carbon Monoxide Measuring Range",
  "Card Slot Type",
  "Card Technology",
  "Carriage Finish",
  "Carriage Hole Size",
  "Carriage Material",
  "Carriage Travel",
  "Cart Color",
  "Cartridge Capacity",
  "Cartridge Location",
  "Cartridge Size",
  "Cartridge Size Compatibility",
  "Cartridge Type",
  "Case Color",
  "Case Height",
  "Case Pressure Rating",
  "Case Size",
  "Case Style",
  "Case Thread",
  "Case Type",
  "Case Width",
  "Caster Material",
  "Caster Size",
  "Caster Type",
  "Catch Length",
  "Catch Width",
  "Category Type",
  "Ceiling Opening",
  "Cell Type",
  "CEMA Rating",
  "Center Distance",
  "Center Distance of Rolls",
  "Center Height",
  "Center Hole Size",
  "Center Hole Thread Size",
  "Center Line Length",
  "Center Line to Base",
  "Center Offset",
  "Center Opening Diameter",
  "Center Span Load",
  "Center to Center",
  "Center to Center Roller Spacing",
  "Center to Cutting Edge",
  "Center to Cutting Height",
  "Centerline Angle",
  "Centerline Height",
  "Centerline to Body",
  "Centerline to Handle Edge",
  "Centre Distance",
  "Chain Grade",
  "Chain Length",
  "Chain Material",
  "Chain Number",
  "Chain Pitch",
  "Chain Size",
  "Chain Width",
  "Chain/Rope Size",
  "Chalk Capacity",
  "Chamfer",
  "Chamfer Angle",
  "Chamfer Diameter",
  "Chamfer Length",
  "Chamfer Size",
  "Chamfer Type",
  "Channel Bandwidth",
  "Channel Height",
  "Channel Length",
  "Channel Mounting Holes",
  "Channel Position",
  "Channel Scan",
  "Channel Size",
  "Channel Spacing",
  "Channel Type",
  "Channel Width",
  "Character",
  "Character Color",
  "Character Height",
  "Character Sets",
  "Character Size",
  "Character Type",
  "Character Width",
  "Character/Legend",
  "Charge by AC Outlet",
  "Charge Indicator",
  "Charge Type",
  "Charges Upto",
  "Charging Range",
  "Chart Size",
  "Chassis Type",
  "Check Type",
  "Chemical Base",
  "Chemical Formula",
  "Chemical Name",
  "Chemical Resistance",
  "Chemical Type",
  "Chest Size",
  "Chest Strap Buckle",
  "Chilling Option",
  "Chipbreaker Style",
  "Chisel Capacity",
  "Chisel Center to Fence",
  "Chisel Size",
  "Chisel Style",
  "Choker Hitch Capacity",
  "Chuck Diameter",
  "Chuck Key Size",
  "Chuck Shank Diameter",
  "Chuck Size",
  "Chuck Style",
  "Chuck Taper",
  "Chuck Through Hole Size",
  "Chuck Type",
  "Circle Capacity",
  "Circle Diameter",
  "Circuit Amperage Rating",
  "Circuit Breaker Rating",
  "Circuit Input",
  "Circuit Length",
  "Circuit Output",
  "Circuit Type",
  "Circuits per Whip",
  "Circumference",
  "Clad Dimension",
  "Cladding Diameter",
  "Clamp Bolts Size",
  "Clamp Face",
  "Clamp Length",
  "Clamp Material",
  "Clamp Opening",
  "Clamp Rating",
  "Clamp Screw Compatibility",
  "Clamp Type",
  "Clamping Angle",
  "Clamping Bolt Size",
  "Clamping Capacity",
  "Clamping Depth",
  "Clamping Force",
  "Clamping Height",
  "Clamping Hole Length",
  "Clamping Length",
  "Clamping Pressure",
  "Clamping Range",
  "Clamping System",
  "Class of Fit",
  "Class Rating",
  "Claw Style",
  "Cleaning Path",
  "Cleanout Size",
  "Clearance",
  "Clearance Diameter",
  "Clearance Height",
  "Clearing Path",
  "Cleat Type",
  "Cleave Length",
  "Cleaving Angle",
  "Clevis Back to Pin O.C.",
  "Clevis Bolt Diameter",
  "Clevis Hole Diameter",
  "Clevis Mounting Type",
  "Clevis Opening",
  "Clevis Pin Diameter",
  "Clevis Width",
  "Climbing Angle",
  "Clip Capacity",
  "Clip Finish",
  "Clip Material",
  "Clip Position",
  "Clip Size",
  "Clip Type",
  "Close Pressure Rating",
  "Closed Depth",
  "Closed Diameter",
  "Closed Hasp Hole Size",
  "Closed Height",
  "Closed Length",
  "Closed Width",
  "Closure Size",
  "Clutch Style",
  "Coarseness of Cut",
  "Coat Type",
  "Coating",
  "Coating Coverage",
  "Coating Diameter",
  "Coating Material",
  "Coating Process",
  "Coating Thickness",
  "Code Base",
  "Coefficient of Volume",
  "Coil Current",
  "Coil Enclosure",
  "Coil Length",
  "Coil Power Rating",
  "Coil Resistance",
  "Coil Termination",
  "Coil Type",
  "Coil Width",
  "Cold Off Temperature",
  "Collapsed Dimensions",
  "Collar Color",
  "Collar Diameter",
  "Collar Height",
  "Collar Length",
  "Collar Material",
  "Collar Size",
  "Collar Style",
  "Collar Thickness",
  "Collation Angle",
  "Collation Type",
  "Collection",
  "Collection Bag Capacity",
  "Collector Bag Length",
  "Collet Capacity",
  "Collet Nut Diameter",
  "Collet Series",
  "Collet Size",
  "Collet System",
  "Color Code",
  "Column Diameter",
  "Commercial/Residential",
  "Communication Interface",
  "Communication Method",
  "Communication Protocol",
  "Communication Type",
  "Compacting Force",
  "Compartment Depth",
  "Compartment Height",
  "Compartment Size",
  "Compartment Width",
  "Compartments per Drawer",
  "Compatibility Notes",
  "Compatible Bearing Type",
  "Compatible Bore Diameter",
  "Compatible Hole Diameter",
  "Compatible Pilot",
  "Compatible Stop Screw",
  "Compatible Taper",
  "Compatible Tool Material",
  "Compatible With",
  "Compensation Range",
  "Compliance Type",
  "Component Mounting",
  "Components Material",
  "Composition",
  "Compound Travel",
  "Compressor",
  "Compressor Oil",
  "Compressor Performance",
  "Compressor Type",
  "Concentration",
  "Concentricity",
  "Conductivity",
  "Conductivity Accuracy",
  "Conductivity Range",
  "Conductor Category",
  "Conductor Stranding",
  "Conductor Strands",
  "Conduit Capacity",
  "Conduit Connection",
  "Conduit Length",
  "Conduit Mounting Type",
  "Conduit Type",
  "Conduit Width",
  "Conduit/Cable Size",
  "Conduit/Pipe Size",
  "Cone Color",
  "Cone Material",
  "Cone Width",
  "Configuration Type",
  "Connecting Blocks",
  "Connection Angle",
  "Connection Location",
  "Connection Material",
  "Connection Method",
  "Connectivity",
  "Connector",
  "Connector Configuration",
  "Connector Impedance",
  "Connector Range",
  "Constant Torque Speed Range",
  "Contact",
  "Contact / Noncontact",
  "Contact Action",
  "Contact Angle",
  "Contact Connection Size",
  "Contact Connection Type",
  "Contact Point Length",
  "Contact Point Type",
  "Contact Rating",
  "Contact Size",
  "Contact Termination",
  "Container Color",
  "Container Material",
  "Container Shape",
  "Container Size Range",
  "Contents",
  "Continuity",
  "Continuous Temperature",
  "Control Air Pressure Rating",
  "Control Cable Length",
  "Control Circuit Type",
  "Control Input",
  "Control Method",
  "Control Mode",
  "Control Period",
  "Control Range",
  "Control Voltage",
  "Controller",
  "Controller Size",
  "Controls/Adjustments",
  "Controls/Indicators",
  "Conversion Type",
  "Convex or Concave Bit",
  "Conveyor Diameter",
  "Conveyor Height Adjustment Range",
  "Cooking Surface Material",
  "Cooktop Output",
  "Coolant Hole Configuration",
  "Coolant Hole Thread",
  "Coolant Line Length",
  "Coolant Type",
  "Cooling Capacity",
  "Cooling Method",
  "Cooling SEER",
  "Cooling System",
  "Cooling Time",
  "Cooling Type",
  "COP Rating",
  "Copper Connectors",
  "Copper Range",
  "Cord Color",
  "Cord Material",
  "Core",
  "Core Diameter",
  "Core Dimension",
  "Core Material",
  "Core Size",
  "Corner Type",
  "Corners",
  "Corrosion-Resistant",
  "Countersink Size Compatibility",
  "Counts",
  "Coupler Type",
  "Coupling",
  "Coupling End Size",
  "Coupling Length",
  "Coupling Material",
  "Coupling Size",
  "Coupling Time",
  "Course Duration",
  "Cover",
  "Cover Bolt Type",
  "Cover Material",
  "Cover Opening Size",
  "Cover Size",
  "Cowl Depth",
  "Cracking Pressure",
  "Crankcase Type",
  "Cranking Amperage",
  "Crankshaft Material",
  "Creepage",
  "Creepage Rating",
  "Crest Factor",
  "Crimp Time",
  "Crimping Length",
  "Crimping Range",
  "Critical Current Rate of Rise",
  "Critical Temperature",
  "Cross Bar Spacing",
  "Cross Section",
  "Cross Section Shape",
  "Cross Travel",
  "Crossarm Mounting Hole Size",
  "Crossarm Size",
  "Crossarm Width",
  "Crosstalk",
  "Crown Size",
  "Crown Width",
  "Crystal Insert Size",
  "CT Ratio",
  "Cubic Capacity",
  "Cuff Style",
  "Cup Width",
  "Cups per Hour",
  "Current Burden",
  "Current Consumption",
  "Current Measuring Range",
  "Current Range",
  "Current Rating",
  "Current/Amps",
  "Cushion Material",
  "Cushion Size",
  "Cushion Type",
  "Cushions",
  "Cut Off Length",
  "Cut Out Height",
  "Cut Out Length",
  "Cut Out Size",
  "Cut Out Width",
  "Cut Profile",
  "Cut Size",
  "Cut Thickness",
  "Cut Type",
  "Cutoff Allowance",
  "Cuts per Minute",
  "Cuts Ties",
  "Cutter Diameter",
  "Cutter Head Diameter",
  "Cutter Head Length",
  "Cutter Head Speed",
  "Cutter Head Type",
  "Cutter Included",
  "Cutter Number",
  "Cutter Speed",
  "Cutter Type",
  "Cutting Accuracy",
  "Cutting Angle",
  "Cutting Application",
  "Cutting Capacity",
  "Cutting Capacity Rip Left",
  "Cutting Capacity Rip Right",
  "Cutting Condition",
  "Cutting Cycle",
  "Cutting Diameter",
  "Cutting Edge Length",
  "Cutting Edge Style",
  "Cutting Force",
  "Cutting Height",
  "Cutting Length",
  "Cutting Radius",
  "Cutting Thickness",
  "Cutting Width",
  "Cycle Life",
  "Cycle Time",
  "Cylinder Action",
  "Cylinder Bore Size",
  "Cylinder Diameter",
  "Cylinder Linear Force",
  "Cylinder Material",
  "Cylinder Stroke",
  "D Ring",
  "Damper Size",
  "Damper Type",
  "Damping Factor",
  "Dart Type",
  "Dash Number",
  "Dash Size",
  "Data Hold",
  "Data Import Facility",
  "Data Storage Capacity",
  "Day/Night Operation",
  "Decibel Rating",
  "Decibel Reduction",
  "Deck Size",
  "Deck Thickness",
  "Deflection",
  "Deflection Angle",
  "Defrost System",
  "Defrost Time",
  "Degree of Protection",
  "Density",
  "Density Grade",
  "Density Number",
  "Density/Hardness",
  "Depth Adjustment",
  "Depth of Bowl",
  "Depth of Cut",
  "Depth Range",
  "Depth Reading",
  "Depth with Door Open",
  "Desiccant Material",
  "Design",
  "Detectable Materials",
  "Detectable Speed",
  "Detection Range",
  "Detection Zones",
  "Detector Tip Style",
  "Detects",
  "Detergent Injection Type",
  "Device Opening",
  "Device Size",
  "Device Type",
  "Dew Point",
  "Dew Point Range",
  "Dial",
  "Dial Color",
  "Dial Diameter",
  "Dial Range",
  "Dial Reading",
  "Dial Size",
  "Dial Type",
  "Diameter Range",
  "Diameter Tolerance",
  "Diameter Under Flange",
  "Diametral Pitch",
  "Diamond Carat Size",
  "Diamond Depth",
  "Diaphragm Type",
  "Die Finish/Coating",
  "Die Head",
  "Die Head Compatibility",
  "Die Head Opening Type",
  "Die Material",
  "Die Outside Diameter",
  "Die Outside Diameter Compatibility",
  "Die Shape",
  "Die Shape Compatibility",
  "Die Size",
  "Die Thread Size",
  "Die Type",
  "Differential Pressure Range",
  "Diffuser Material",
  "Diffuser Shape",
  "Digit Size",
  "Digital Video Recorder Size",
  "Dilution",
  "Dimensions",
  "Dimmable",
  "Dimming Rate",
  "DIN Size",
  "DIN Type",
  "Diode Test",
  "Diopter",
  "Direction",
  "Direction of Rotation",
  "Disc Diameter",
  "Disc Material",
  "Disc Thickness",
  "Discharge Connection Type",
  "Discharge Diameter",
  "Discharge Dimensions",
  "Discharge Length",
  "Discharge Line Size",
  "Discharge Option",
  "Discharge Pipe Size",
  "Discharge Pressure",
  "Discharge Size",
  "Discharge Type",
  "Discharge Width",
  "Disconnect",
  "Dispenses Type",
  "Dispensing Force",
  "Dispersion",
  "Displacement",
  "Display Color",
  "Displayed Temperature Rating",
  "Disposable/Reusable",
  "Dissipation Constant",
  "Distance",
  "Distance Between Blade Tips",
  "Distance Between Centers",
  "Distance Between Coil",
  "Distance Between Flanges",
  "Distance Between Hooks",
  "Distance Between Mount Hole Centers",
  "Distance Between Mounting Holes",
  "Distance Between Roller Ends",
  "Distance Between Shafts",
  "Distance Between Shelves",
  "Distance Between Yokes",
  "Distance from Pole",
  "Distance Range",
  "Distance Supports",
  "Distortion",
  "Distribution Blocks",
  "Divider Direction",
  "Dividers per Drawer",
  "Dividing Accuracy",
  "Division Degree Graduations",
  "DOE Heating Capacity",
  "Dome Diameter",
  "Dome Material",
  "Door",
  "Door Handle",
  "Door Height",
  "Door Latching",
  "Door Leaf Length",
  "Door Leaf Width",
  "Door Size",
  "Door Station Size",
  "Door Thickness",
  "Door Type",
  "Door Width",
  "Dot Pitch/Pixel Pitch",
  "Double Capacity",
  "Double Shear Minimum",
  "Down Rod Size",
  "Draft Depth",
  "Drain Connection",
  "Drain Height",
  "Drain Hole Diameter",
  "Drain Line Capacity",
  "Drain Line Size",
  "Drain Location",
  "Drain Material",
  "Drain Opening Size",
  "Drain Outlet Size",
  "Drain Pipe Size",
  "Drain Rate",
  "Drain Size",
  "Drain Type",
  "Draw-off Length",
  "Drawbar Thread Size",
  "Drawer Capacity",
  "Drawer Dimensions",
  "Drawer Material",
  "Drawer Size",
  "Drill Bit Cutting Shape",
  "Drill Bit Function",
  "Drill Bit Size Compatibility",
  "Drill Bit Style",
  "Drill Blank Size",
  "Drill Body Length",
  "Drill Body Length to Flange",
  "Drill Body Size",
  "Drill Depth",
  "Drill Depth by Diameter Ratio",
  "Drill Diameter",
  "Drill Diameter Measurement System",
  "Drill Diameter Range",
  "Drill Finish/Coating",
  "Drill Length",
  "Drill Material",
  "Drill Point Diameter",
  "Drill Point Length",
  "Drill Point Pressure",
  "Drill Shank Diameter",
  "Drill Size",
  "Drill Size - Fraction",
  "Drill Size - Letter",
  "Drill Size - Metric",
  "Drill Size - Wire",
  "Drill Size Compatibility",
  "Drill Style",
  "Drill to Center From Base",
  "Drill Type",
  "Drilling Application",
  "Drilling Capacity",
  "Drive Bays",
  "Drive End Width",
  "Drive Gears",
  "Drive Motor",
  "Drive Rod Thread Size",
  "Drive Size",
  "Drive Voltage",
  "Drive/Shank Size",
  "Driver Type",
  "Drivers Included",
  "Drop",
  "Drop Type",
  "Drum Capacity",
  "Drum Diameter",
  "Drum Gauge",
  "Drum Length",
  "Drum Material",
  "Drum Opening",
  "Drum Speed",
  "Dry Weight",
  "Dryer Capacity",
  "Drying Time",
  "Duct Diameter",
  "Duct Height",
  "Duct Opening",
  "Duct Size",
  "Duct Spacing",
  "Duct Type",
  "Duct Width",
  "Durability",
  "Durometer",
  "Durometer Rating",
  "Dust Collector Included",
  "Dust Holding Capacity",
  "Dust Management",
  "Dust Weight Arrestance",
  "Duty",
  "Duty Cycle Voltage",
  "Duty Rating",
  "Earth Voltage",
  "Edge Profile",
  "Edge Thickness",
  "Edge Type",
  "Edition of Publication",
  "EER Rating",
  "Effective Area",
  "Effective Audible Range",
  "Effective Height",
  "Effective Length",
  "Effective Range",
  "Elasticity/Stretch",
  "Elastomers Material",
  "Electric Strength",
  "Electrical Durability",
  "Electrical Output",
  "Electrical Specifications",
  "Electronic Feedback LCD",
  "Electronic Shutter Speed",
  "Element Width",
  "Elongation",
  "Elongation at Break",
  "Emissions",
  "Emissivity",
  "Emitter Type",
  "Enclosure Capacity",
  "Enclosure Color",
  "Enclosure Material",
  "Enclosure Mounting Type",
  "Enclosure Rating",
  "Enclosure Size",
  "End A",
  "End A Type",
  "End B",
  "End B Type",
  "End Configuration",
  "End Connection Type",
  "End Fitting Type",
  "End Opening Width",
  "End Shape",
  "End Size",
  "End Support Type",
  "End Vacuum",
  "End/End",
  "Endurance",
  "Endurance Limit",
  "Energy Factor",
  "Energy Source",
  "Engaged Thickness",
  "Engine Displacement",
  "Engine Specifications",
  "Engine Speed",
  "Engine Starter Type",
  "Engine Type",
  "Entrance Cap",
  "Entry Position",
  "Entry Type",
  "Environmental Compatibility",
  "Equalizer Type",
  "Equipment Mounting Hardware",
  "Equipment Size",
  "Equipment Type",
  "Escape Duration",
  "ESD-Safe",
  "Estimated Sound Circle",
  "Evaporative Rate",
  "Exhaust",
  "Exhaust Capacity",
  "Exhaust Hose Length",
  "Exhaust Location",
  "Expanded Area",
  "Expanded Insulation Diameter",
  "Expanded Width",
  "Expansion Tool Pipe Capacity",
  "Expansion Type",
  "Expansion/Connectivity",
  "Exposure Time",
  "Extended Length",
  "Extension Bar Stock Size",
  "Extension Cable Length",
  "Extension Diameter",
  "Extension Length",
  "Extension Size",
  "Extension/Retraction Ratio",
  "Exterior Thread Size",
  "Exterior Type",
  "External Holding Force",
  "External Size",
  "External Thread Size",
  "External Thread Use",
  "Extractor Size",
  "Eye End Size",
  "Eye Inside Diameter",
  "Eye Opening Size",
  "Eye Outside Diameter",
  "Eye Shape",
  "Eye Size",
  "Eye Wash Handle Material",
  "Eye Width",
  "Eyelet Size",
  "Eyelet Thread Size",
  "Eyepiece Protractor",
  "Fabric Material",
  "Fabric Weight",
  "Face Angle",
  "Face Area",
  "Face Color",
  "Face Diameter",
  "Face Diameter Range",
  "Face Length",
  "Face Material",
  "Face Plate Diameter",
  "Face Surface",
  "Face to Face Width",
  "Face Type",
  "Face Width",
  "Facepiece Material",
  "Fan Blade",
  "Fan Connector Type",
  "Fan Motor Wattage",
  "Fan Size",
  "Fan Speed",
  "Fan Speed Control",
  "Fan Switching",
  "Fan Type",
  "Fastener Length",
  "Fastener Material",
  "Fastener Size",
  "Fastener Type",
  "Fastener Type Compatibility",
  "Fasteners",
  "Fastening Method",
  "Faucet Centers",
  "Faucet Finish",
  "Faucet Handle Type",
  "Faucet Hole Center Spacing",
  "Faucet Hole Drilling",
  "Faucet Hole Size",
  "Faucet Hole Spacing",
  "Faucet Location",
  "Faucet Mount Type",
  "Faucet Size",
  "Faucet Type",
  "Fault Protection",
  "Feed Rate",
  "Feed Speed",
  "Feed Type",
  "Feeder Size",
  "Feeding Cable Length",
  "Feeding Force",
  "Feeding Speed",
  "Feeler Length",
  "Female Drive Size",
  "Fence Positive Stops",
  "Fence Size",
  "Fence Type",
  "Ferrule Material",
  "Ferrule Size",
  "Ferrule Type",
  "Fiber Capacity",
  "Fiber Coating Diameter",
  "Fiber Connectors",
  "Fiber Count",
  "Fiber Diameter",
  "Fiber Range",
  "Fiber Size",
  "Fiber Type",
  "Field Connection Type",
  "Field of View",
  "Filament Material",
  "Filament/Wire Diameter",
  "Filament/Wire Type",
  "File Color",
  "File Finish",
  "File Material",
  "File Size",
  "File Size Compatibility",
  "Fill Material",
  "Fill Type",
  "Filled/Unfilled",
  "Filler Valve",
  "Filling Rate",
  "Fills Type",
  "Film Size",
  "Filter",
  "Filter Bag Diameter",
  "Filter Bag Length",
  "Filter Bag Size",
  "Filter Class",
  "Filter Efficiency",
  "Filter Material",
  "Filter Media",
  "Filter Performance",
  "Filter Rating",
  "Filter Size",
  "Filter Strainer",
  "Filter Type",
  "Filtered Contaminants",
  "Filters and Cartridges Type",
  "Fin Material",
  "Fin Size",
  "Fin Type",
  "Final End Force",
  "Final Resistance",
  "Finger Guard",
  "Finish/Coating",
  "Finish/Color",
  "Finished Size",
  "Fire Rating",
  "Firewall Mode of Operation",
  "Firewall Security",
  "Firing Action",
  "Firing Rate",
  "Fit Test Protocol",
  "Fits",
  "Fits Axle",
  "Fits Beam Flange Width",
  "Fits Beam Height",
  "Fits Belts Up To",
  "Fits Bowl Size",
  "Fits Duct Size",
  "Fits Fan Diameter",
  "Fits Groove Diameter",
  "Fits Hammers",
  "Fits Hat Size",
  "Fits Laptop Size",
  "Fits Maximum Handle Size",
  "Fits Maximum Hat Size",
  "Fits Maximum Pipe Size",
  "Fits Minimum Handle Size",
  "Fits Minimum Hat Size",
  "Fits Minimum Pipe Size",
  "Fits Morse Taper Sockets/Sleeves",
  "Fits Pipe Diameter",
  "Fits Shaft Size",
  "Fits Shoe Size",
  "Fits Waist Size",
  "Fits Wrist Size",
  "Fitting Material",
  "Fitting Size",
  "Fitting Threads",
  "Fitting Type",
  "Fixed/Variable",
  "Fixing Type",
  "Fixture Size",
  "Fixture Unit Capacity",
  "Flame Temperature",
  "Flammability",
  "Flammability Rating",
  "Flange Diameter",
  "Flange Length",
  "Flange Mounting Type",
  "Flange Outside Diameter",
  "Flange Range",
  "Flange Thickness",
  "Flange to Angle Length",
  "Flange to Knob Length",
  "Flange Type",
  "Flange Width",
  "Flare Angle",
  "Flare Type",
  "Flash Point",
  "Flashes per Minute",
  "Flat Contact Diameter",
  "Flatness",
  "Flatness Unilateral Tolerance",
  "Flavor",
  "Flexibility",
  "Flight Thickness",
  "Flight Type",
  "Float Diameter",
  "Float from Center",
  "Float Height",
  "Float Length",
  "Float Material",
  "Float Shape",
  "Float Size",
  "Float Specific Gravity",
  "Float Weight",
  "Float Width",
  "Floor Base Size",
  "Floor Opening",
  "Floor Type",
  "Flow Capacity",
  "Flow Control",
  "Flow Measuring Range",
  "Flow Rating",
  "Flow Type",
  "Flue Outlet Size",
  "Flue Size",
  "Fluid Capacity",
  "Fluid Delivery Speed",
  "Fluid Delivery Type",
  "Fluid Port Size",
  "Fluid Quantity",
  "Fluid Temperature Rating",
  "Fluid Type",
  "Fluid Volume",
  "Fluids Absorbed",
  "Flush Handle",
  "Flush Rate",
  "Flush Valve Size",
  "Flushing System",
  "Flute Direction",
  "Foam Density",
  "Focal Length",
  "Focal Type",
  "Focus Spot Size",
  "Focusing Height",
  "Focusing Travel",
  "Folded Dimensions",
  "Folding Depth",
  "Font Size",
  "Foot Plate Size",
  "Foot Type",
  "For Blade Shank Type",
  "For Bolt Size",
  "For Caster Diameter",
  "For Chuck Size",
  "For Conduit/Pipe Diameter",
  "For Fastener Type",
  "For Gas Type",
  "For Hole Size",
  "For Max Backing Pad Diameter",
  "For Max Mild Steel Thickness",
  "For Max Stainless Steel Thickness",
  "For Max Wire Thickness",
  "For Maximum Cutter Hole Diameter",
  "For Maximum EMT Diameter",
  "For Maximum IMC Diameter",
  "For Maximum Pipe Diameter",
  "For Maximum Tube Diameter",
  "For Maximum Wheel Diameter",
  "For Maximum Wire Thickness",
  "For Minimum Cutter Hole Diameter",
  "For Panel Range",
  "For Pipe Diameter",
  "For Pipe Size",
  "For Post Size",
  "For Rivet Size",
  "For Screw Diameter",
  "For Screw Size",
  "For Shaft Diameter",
  "For Strapping Material",
  "For Strapping Width",
  "For Stringers",
  "For Tool Post Type",
  "For Use On",
  "For Wheel Type",
  "For Wire Thickness",
  "Force",
  "Fork Height Lowered",
  "Fork Height Raised",
  "Fork Length",
  "Fork Width",
  "Form Factor",
  "Format",
  "Formula",
  "Forward Current",
  "Forward Voltage",
  "Forward Voltage Drop",
  "Forwarding Rate",
  "Fragrance",
  "Frame Color",
  "Frame Finish",
  "Frame Finish/Color",
  "Frame Leaf Length",
  "Frame Leaf Width",
  "Frame Material",
  "Frame Size",
  "Free Coil Diameter",
  "Free Speed",
  "Freeboard Height",
  "Freezer Capacity",
  "Freezing Point",
  "Frequency",
  "Frequency Counter",
  "Frequency Flashover",
  "Frequency Resolution",
  "Front Panel",
  "Front Type",
  "Front Wheel Diameter",
  "Front Wheel Width",
  "Fuel Consumption",
  "Full Load Amperage",
  "Functional Cure",
  "Functions With",
  "Furnace Position",
  "Fuse",
  "Fuse Block Size",
  "Fuse Clip Rating",
  "Gage Insert",
  "Gage Length",
  "Gage Length/Projection",
  "Gain",
  "Gallons",
  "Gallons per Flush",
  "Gangs",
  "Garment Max Arc Flash Protection",
  "Garment Type",
  "Gas Connection Size",
  "Gas Inlet Size",
  "Gas Supply Size",
  "Gas Type",
  "Gas Volume",
  "Gasket Material",
  "Gasket Type",
  "Gate Clearance",
  "Gate Trigger Current",
  "Gate Trigger Voltage",
  "Gauge",
  "Gauge Diameter",
  "Gauge Glass Type",
  "Gauge Port Size",
  "Gauge Range",
  "Gauge Thickness",
  "Gauge Type",
  "Gauge/Conductor",
  "Gear Type",
  "Gender",
  "Geometry",
  "Glass Stage Area",
  "Glass Style",
  "Glass Thickness",
  "Glass Type",
  "Glove Material Thickness",
  "Glove Type",
  "Go/No Go",
  "Gong Size",
  "Graduation",
  "Graduation Type",
  "Graduations",
  "Graphics",
  "Grate Material",
  "Grate Type",
  "Grease Filler Material",
  "Grease Filler Type",
  "Grid Length",
  "Grid Material",
  "Grid Size",
  "Grid Thickness",
  "Grid Width",
  "Grille Color",
  "Grille Dimensions",
  "Grille Length",
  "Grille Width",
  "Grinding Chamber Capacity",
  "Grinding Speed",
  "Grip Material",
  "Grip Range",
  "Grip Style",
  "Grip Type",
  "Grit",
  "Grommet Material",
  "Grommet Size",
  "Grommet Type",
  "Groove Angle",
  "Groove Depth",
  "Groove Diameter",
  "Groove Edge Width",
  "Groove Height",
  "Groove Radius",
  "Groove Size",
  "Groove Width",
  "Groove/Plug Type",
  "Gross Area",
  "Gross Torque",
  "Ground Connection",
  "Grounding Style",
  "Guard Length",
  "Guard Type",
  "Guide Bar Length",
  "Guide Function",
  "Guy Attachment",
  "Guy Opening",
  "Guy Wire Size",
  "Half Power Beam Width (HPBW)",
  "Hammer Type",
  "Hammer Weight Range",
  "Hand of Holder",
  "Hand of Insert",
  "Hand of Tool",
  "Hand Shower Yes/No",
  "Hand Type",
  "Hand Wheel Diameter",
  "Handguard",
  "Handle Bend Type",
  "Handle Clamp",
  "Handle Diameter",
  "Handle End Thread Size",
  "Handle Grip Material",
  "Handle Height",
  "Handle Length",
  "Handle Shape",
  "Handle Style",
  "Handle Width",
  "Handrail Height",
  "Handset Type",
  "Hard Disk",
  "Hard Drive Capacity",
  "Hard Drive Size",
  "Hard Start Timing",
  "Hardness Depth",
  "Hardware Compatibility",
  "Hardware Material",
  "Hardware Requirements",
  "Harmonic Distortion",
  "Harmonic Range",
  "Harness Connection Hook Type",
  "Hazard Code",
  "Hazard Risk Category (HRC)",
  "Head Angle",
  "Head Coating",
  "Head Color",
  "Head Connection Size",
  "Head Diameter",
  "Head Drive Type",
  "Head Finish",
  "Head Height",
  "Head Holder Mechanism",
  "Head Length",
  "Head Length/Projection",
  "Head Material",
  "Head Shape",
  "Head Size",
  "Head Thickness",
  "Head Travel",
  "Head Vertical Travel",
  "Head Weight",
  "Head Width",
  "Head/Face Protection Type",
  "Head/Holder Diameter",
  "Headband Size",
  "Header",
  "Headgear",
  "Headgear Color",
  "Headgear Material",
  "Headgear Style",
  "Headroom",
  "Headroom to Bail",
  "Headroom to Shaft",
  "Headstock ID Taper Size",
  "Headstock Taper Size",
  "Headstrap Type",
  "Heat Input",
  "Heat Pump Compatible",
  "Heat Resistance",
  "Heater Type",
  "Heater Wattage",
  "Heating Area",
  "Heating Cable Spacing",
  "Heating Element",
  "Heating Time",
  "Height Adjustable Range",
  "Height Adjustment",
  "Height Range",
  "Height to Center Line",
  "Height Under Arm",
  "Height with Lid Open",
  "Helix Diameter",
  "Helix Size",
  "Helix Spacing",
  "Hex Key Size",
  "Hex Size Range",
  "Hex Width",
  "Hex Width Tolerance",
  "High Pressure Tube Diameter",
  "High Speed",
  "High Voltage Detection",
  "High Voltage Test",
  "High-Visibility",
  "Hinge Material",
  "Hinge Size",
  "Hinge Type",
  "Hinged",
  "Hoist Base",
  "Hoist Type",
  "Hold Up Time",
  "Holder Compatibility",
  "Holder Shank Width",
  "Holder Type",
  "Holder/Bar Length",
  "Holding Capacity",
  "Holding Current",
  "Holding Force",
  "Holding Strength",
  "Hole Cable Diameter",
  "Hole Chamfer Type",
  "Hole Chamfering",
  "Hole Depth",
  "Hole Diameter Using TPGH 321",
  "Hole Diameter Using TPGH 322",
  "Hole Saw Diameter",
  "Hole Sizes Cut",
  "Hole Thread Size",
  "Hole to Hole Length",
  "Holes per Leaf",
  "Hood Length",
  "Hood Opening",
  "Hood Size",
  "Hood Style",
  "Hood Type",
  "Hood Width",
  "Hood/Face Shield Max Arc Flash Protection",
  "Hooded (Yes/No)",
  "Hook Angle",
  "Hook Length",
  "Hook Material",
  "Hook Mounting Type",
  "Hook Opening",
  "Hook Size",
  "Hook Type",
  "Hopper Material",
  "Horizontal Bus Rating",
  "Horizontal Graduations",
  "Horizontal Range",
  "Horn Length",
  "Horn Material",
  "Hose Capacity",
  "Hose Connection",
  "Hose Connection Size",
  "Hose Diameter",
  "Hose Dimensions",
  "Hose Inside Diameter",
  "Hose Length",
  "Hose Material",
  "Hose Outside Diameter",
  "Hose Size",
  "Hose Type",
  "Host Bus Interface",
  "Host Interface",
  "Hours on One Filling",
  "Housing Bore Size",
  "Housing Color",
  "Housing Diameter",
  "Housing Dimensions",
  "Housing Height",
  "Housing Length",
  "Housing Size",
  "Housing Type",
  "Housing Width",
  "HSPF Rating",
  "Hub Center-to-Center",
  "Hub Contact Pressure",
  "Hub Diameter",
  "Hub Flange Diameter",
  "Hub Length",
  "Hub Location",
  "Hub Outside Diameter",
  "Hub Projection",
  "Hub Width",
  "Humidity Accuracy",
  "Humidity Range",
  "Humidity Rating",
  "Humidity Removal Capacity",
  "Humidity Sensing",
  "Hydraulic System Type",
  "Ice Cube Size",
  "Ice Shape",
  "ID Clamping Range",
  "IEER Rating",
  "Igniter Warm-Up Time",
  "Ignition Type",
  "Illumination",
  "Image Acquisition",
  "Image Aspect Ratio",
  "Image Brightness",
  "Image Color Temperature",
  "Image Contrast Ratio",
  "Image Sensor Format",
  "Image Support",
  "Imager",
  "Immersion Depth",
  "Impact Energy",
  "Impact Rated",
  "Impact Strength",
  "Impedance",
  "Impedance Range",
  "Impeller Diameter",
  "Impeller Material",
  "Impeller Type",
  "Impression Letter Type",
  "Impulse Flashover",
  "Impulse Withstand Voltage",
  "In Solution Shelf Life",
  "Included Angle",
  "Included Socket Size",
  "Includes Drill",
  "Includes Installation Tool",
  "Includes Lift Rod",
  "Includes Tang Removal Tool",
  "Includes Tap",
  "Increments",
  "Index Drive",
  "Indexable Mill Compatibility",
  "Indexable Tool Type",
  "Indexing Accuracy",
  "Indicating Type",
  "Indication Type",
  "Indoor/Outdoor",
  "Inductance",
  "Inductance Accuracy",
  "Inductance Range",
  "Inductance Type",
  "Inductive Rating",
  "Industry Number",
  "Industry Specification",
  "Industry Standard Number",
  "Industry Standards",
  "Infrared Capability",
  "Infrared Measurement",
  "Ingress Protection Rating",
  "Initial End Force",
  "Initial Resistance",
  "Ink Color",
  "Inlet Connection",
  "Inlet Diameter",
  "Inlet Fitting",
  "Inlet Height",
  "Inlet Pipe Size",
  "Inlet Port",
  "Inlet Pressure",
  "Inlet Pressure Range",
  "Inlet Size",
  "Inlet Spud Size",
  "Inlet to Outlet Length",
  "Inlet Type",
  "Inlet Valve Type",
  "Inlet Water Temperature Rating",
  "Inlet/Outlet Size",
  "Inner Diameter",
  "Inner Diameter Expanded",
  "Inner Diameter Recovered",
  "Inner Material",
  "Inner Tube Material",
  "Input Amperage",
  "Input Capacity",
  "Input Circuits",
  "Input Conductor",
  "Input Current",
  "Input Current per String",
  "Input Device",
  "Input Frequency",
  "Input Impedance",
  "Input Level",
  "Input Protection",
  "Input Rating",
  "Input Size",
  "Input Supply",
  "Input Terminal",
  "Input Values",
  "Input/Output Communication",
  "Input/Output Type",
  "Inputs",
  "Inrush Current",
  "Inseam Length",
  "Insert Compatibility",
  "Insert Cutting Radius",
  "Insert Diameter",
  "Insert Gage Screw",
  "Insert Height",
  "Insert Holding Method",
  "Insert Length",
  "Insert Number",
  "Insert Opening",
  "Insert Screw",
  "Insert Seat Size",
  "Insert Shape",
  "Insert Thickness",
  "Insert Thread Size",
  "Insert Type",
  "Insert Width",
  "Insertion Depth",
  "Insertion Length",
  "Insertion Loss",
  "Inside Depth",
  "Inside Diameter Tolerance",
  "Inside Dimensions",
  "Inside Eye Diameter",
  "Inside Height",
  "Inside Hole Diameter",
  "Inside Hub Size",
  "Inside Length",
  "Inside Modular Connection Size",
  "Inside Morse Taper Size",
  "Inside Square Size",
  "Inside Taper Size",
  "Inside Width",
  "Inspection Light",
  "Installation",
  "Installation Size",
  "Installation Torque",
  "Installation Type",
  "Insulated (Yes/No)",
  "Insulated Grip",
  "Insulated Length",
  "Insulated/Non-Insulated",
  "Insulating Resistance",
  "Insulation Capacity",
  "Insulation Color",
  "Insulation Depth",
  "Insulation Diameter",
  "Insulation Outer Diameter",
  "Insulation Rating",
  "Insulation Resistance",
  "Insulation Resistance Range",
  "Insulation Resistance Test Voltage",
  "Insulation Stripping Length",
  "Insulation Thickness",
  "Insulation Value",
  "Insulator Class",
  "Insulator Length",
  "Insulator Mounting Type",
  "Insulator Opening",
  "Insulator Size",
  "Insulator Thread Length",
  "Insulator Type",
  "Integrated Light",
  "Intensity Adjustment",
  "Interconnection Capability",
  "Interface Connection",
  "Interface/Ports",
  "Interior Thread Size",
  "Interior Type",
  "Interleaf",
  "Intermitter Temperature",
  "Internal Holding Force",
  "Internal Modulation Mode",
  "Internal or External",
  "Internal Plate",
  "Internal Thread Size",
  "Internal Thread Use",
  "Intrusion Detection System",
  "ISO Code",
  "Isolated Circuits",
  "Isolation",
  "Isolation Loss",
  "Isolator Type",
  "J-Width",
  "Jack Position",
  "Jacket Outside Diameter",
  "Jacket Rating Code",
  "Jacket Type",
  "Jacobs Equivalent",
  "Jaw Angle",
  "Jaw Bend",
  "Jaw Capacity",
  "Jaw Depth",
  "Jaw Height",
  "Jaw Length",
  "Jaw Opening",
  "Jaw Reach",
  "Jaw Spread",
  "Jaw Surface",
  "Jaw Surface Type",
  "Jaw Thickness",
  "Jaw Width",
  "Jaws",
  "Jet Pressure",
  "Joint Diameter",
  "Jointing Capacity",
  "Joist Size",
  "Jumper Rating",
  "Junction",
  "Junction Temperature",
  "Kerf",
  "Key Capacity",
  "Key Layout",
  "Key Material",
  "Key Number",
  "Key Size",
  "Key Type",
  "Keypad Lighting",
  "Keypad Lock",
  "Keypad Type",
  "Keystroke Tone",
  "Keyway Depth",
  "Keyway Length",
  "Keyway Width",
  "Kit Type",
  "Knee Travel",
  "Knife Size",
  "Knife Type",
  "Knob Adjustment",
  "Knob Diameter",
  "Knob Material",
  "Knockout Inner Diameter",
  "Knurl Diameter",
  "Knurl Face Width",
  "Knurl Hole Diameter",
  "Label Legend",
  "Label Length",
  "Label Type",
  "Label Width",
  "Labels per Cartridge",
  "Labels per Sheet",
  "Lacing",
  "Ladder Height",
  "Lamp Base",
  "Lamp Capacity",
  "Lamp Length",
  "Lamp Life",
  "Lamp Mounting",
  "Lamp Size",
  "Lamp Test",
  "LAN Category",
  "Language",
  "Large End Diameter",
  "Laser Beam Angle",
  "Laser Class",
  "Laser Range",
  "Laser Sighting",
  "Laser Wavelength",
  "Latch Length",
  "Latch Type",
  "Latching Current",
  "Lathe Swing",
  "Lathers Channel Height",
  "Layer Material",
  "Layer Structure",
  "LCM Module",
  "Lead",
  "Lead Angle",
  "Lead Angle Direction",
  "Lead Count",
  "Lead Cross Section",
  "Lead Diameter",
  "Lead In Diameter",
  "Lead Length",
  "Lead Material",
  "Lead Size",
  "Lead Type",
  "Leaf Thickness",
  "Leakage Distance",
  "LED Indicator",
  "Leg Length",
  "Leg Material",
  "Leg Size",
  "Leg Strap Buckle",
  "Leg Type",
  "Legend Style",
  "Legend/Background Color",
  "Length Accuracy",
  "Length Before Compression",
  "Length of Arm",
  "Length of Tangs",
  "Length of Tines",
  "Length per Pound",
  "Length Through Bore",
  "Length Through Hub",
  "Length to Diameter Ratio",
  "Length Type",
  "Length Under Flange",
  "Length Under Head",
  "Length Under Head/Projection",
  "Length Under Load",
  "Length Under Strand",
  "Lengths Included",
  "Lens",
  "Lens Coating",
  "Lens Control",
  "Lens Diameter",
  "Lens Length",
  "Lens Shade",
  "Lens Width",
  "Letter Height",
  "Letter/Wire Size",
  "Levels of Color",
  "Lever Length",
  "Licensing Period",
  "Lid Color",
  "Lid Depth",
  "Lid Type",
  "Life Hours",
  "Lift",
  "Lift Arms Width",
  "Lift Speed",
  "Lifting Capacity",
  "Lifting Force",
  "Lifting Height",
  "Light Kit",
  "Light Level",
  "Light Level Accuracy",
  "Light Wavelength",
  "Lighting Area",
  "Lighting Type",
  "Line Advance",
  "Line Color",
  "Line Connector Type",
  "Line Diameter",
  "Line Length",
  "Line Material",
  "Line Power",
  "Line Range",
  "Line Rate",
  "Line Resolution",
  "Line Spacing Width",
  "Linear Guide Length",
  "Linear Loading",
  "Linear Motion",
  "Linearity",
  "Liner Length",
  "Liner Material",
  "Lining",
  "Lining Material",
  "Link Plate Height",
  "Link Plate Thickness",
  "Link Type",
  "Lip",
  "Lip Height",
  "Lip Material",
  "Liquid Filled",
  "Liquid Line Size",
  "Liquid Temperature Rating",
  "Listen Channel Output",
  "Load",
  "Load at Deflection",
  "Load at Flat",
  "Load Break Rating",
  "Load Center",
  "Load Current",
  "Load Impedance",
  "Load Life",
  "Load Limit",
  "Local Language",
  "Locate Shorts",
  "Location of Heating Elements",
  "Locator Range",
  "Lock Angle",
  "Lock Body Depth",
  "Lock Body Length",
  "Lock Body Width",
  "Lock Mechanism",
  "Lock Type",
  "Locked Rotor Amperage",
  "Locking Device",
  "Locking Mechanism",
  "Locking System",
  "Locking Type",
  "Lockout Size",
  "Lockout Time",
  "Log Size",
  "Longitudinal Travel",
  "Loop Amperage Rating",
  "Loop Diameter",
  "Loop Resistance",
  "Loop Size",
  "Loop Voltage Rating",
  "Low Battery Indicator",
  "Low Lead Compliance",
  "Low NOX",
  "Lower Hole Diameter",
  "Lower Oven Capacity",
  "Lower Rod Penetration",
  "Lowered Height",
  "Lubricant Line Length",
  "Lubricant Type",
  "Lubrication Hole Type",
  "Lug Length",
  "Lug Material",
  "Lug Size",
  "Lug Type",
  "Luminance Factor",
  "Machine Compatibility",
  "Machine Direction",
  "Machine Type",
  "Machining Process",
  "Magazine Capacity",
  "Magazine Loading",
  "Magnet Material",
  "Magnetic",
  "Magnetic Flux Density",
  "Magnetic Pull",
  "Magnetic Surface Size",
  "Magnetized Tip",
  "Magnification",
  "Magnification Power",
  "Main Lug Amps",
  "Main Lugs",
  "Main Tool Style",
  "Main/Run Size",
  "Mainline Pressure Rating",
  "Major Radius",
  "Male Drive Size",
  "Mandrel Shank Diameter",
  "Mandrel Shank Length",
  "Manifold Material",
  "Manufacturer's Grade",
  "Marker",
  "Marker Color",
  "Marker Legend",
  "Marker Length",
  "Marker Plate Length",
  "Marker Plate Marking Limit",
  "Marker Plate Thickness",
  "Marker Plate Width",
  "Marker Size",
  "Marker Type",
  "Marker Width",
  "Markers per Card",
  "Markings",
  "Masking Plate Divisions",
  "Mast Length",
  "Master Station Size",
  "Material - Body/Mandrel",
  "Material Above the Bail Opening",
  "Material Application",
  "Material Diameter",
  "Material Fastened To",
  "Material Gauge",
  "Material Hardness",
  "Material of Construction",
  "Material Specification",
  "Material Tensile Strength",
  "Material Thickness",
  "Material Type",
  "Mating Ring Material",
  "Mating Ring Outside Diameter",
  "Mating Ring Thickness",
  "Max Acceptance Factor",
  "Max Characters per Line",
  "Max Compatible Temp",
  "Max Compatible Thickness",
  "Max Core Bit Compatibility",
  "Max Pickup Height",
  "Max Pull Capacity",
  "Max Reach",
  "Max Take Off Angle",
  "Max Waist Size",
  "Max Working Torque",
  "Max. Allowable Load",
  "Max. Chuck Length",
  "Max. Diameter Second Coil",
  "Maximum Adjustable Size",
  "Maximum Air Pressure",
  "Maximum Allowable Voltage",
  "Maximum Amperage Rating",
  "Maximum Aperture",
  "Maximum Application Temperature",
  "Maximum Arbor Diameter",
  "Maximum Arc Flash Protection",
  "Maximum Ball Reach",
  "Maximum Bore Depth",
  "Maximum Bore Depth - Largest Bar",
  "Maximum Bore Depth - Smallest Bar",
  "Maximum Bore Diameter",
  "Maximum Box Size",
  "Maximum Bundle Diameter",
  "Maximum Cable Diameter",
  "Maximum Cable Size",
  "Maximum Capacity",
  "Maximum Centerline Height",
  "Maximum Chamfer Diameter",
  "Maximum Code Size",
  "Maximum Counterbore Depth",
  "Maximum Cross Feed Rate per Revolution",
  "Maximum Cut at 45 Deg",
  "Maximum Cut at 90 Deg",
  "Maximum Cutout",
  "Maximum Cutter Diameter",
  "Maximum Cutting Capacity",
  "Maximum Cutting Depth",
  "Maximum Cutting Diameter",
  "Maximum Cutting Tool Size",
  "Maximum Data Access Time",
  "Maximum Data Distance",
  "Maximum Deck Thickness",
  "Maximum Deflection",
  "Maximum Depth",
  "Maximum Depth of Cut",
  "Maximum Diameter",
  "Maximum Diameter Capacity",
  "Maximum Diameter of Dado",
  "Maximum Diameter of Solids",
  "Maximum Discharge Voltage",
  "Maximum Distance",
  "Maximum Distance to Lamp",
  "Maximum Document Width",
  "Maximum Door Height",
  "Maximum Door Width",
  "Maximum Drill Blank Size",
  "Maximum Drill Depth",
  "Maximum Drill Diameter",
  "Maximum Drill Point Diameter",
  "Maximum Drill Size",
  "Maximum Energy",
  "Maximum Expansion",
  "Maximum Extension",
  "Maximum Flow Rate",
  "Maximum Force",
  "Maximum Frames per Second",
  "Maximum Grit",
  "Maximum Grooving Width",
  "Maximum Hardness",
  "Maximum Head",
  "Maximum Head Diameter",
  "Maximum Height",
  "Maximum Height of Pan Sides",
  "Maximum Hole Depth",
  "Maximum Hole Diameter",
  "Maximum Hole Size",
  "Maximum Hose Velocity",
  "Maximum House Fillet Radius",
  "Maximum House Shoulder Diameter",
  "Maximum Immersion Depth",
  "Maximum Included Angle",
  "Maximum Inlet Pressure",
  "Maximum Inlet Temperature",
  "Maximum Input Current",
  "Maximum Instantaneous Energy",
  "Maximum Jaw Opening",
  "Maximum Jaw Spread",
  "Maximum Label Length",
  "Maximum Label Width",
  "Maximum Leg Spread",
  "Maximum Length",
  "Maximum Lifting Height",
  "Maximum Line of Text",
  "Maximum Line Range",
  "Maximum Lines of Print",
  "Maximum Load",
  "Maximum Load Torque",
  "Maximum Longitudinal Feed Rate per Revolution",
  "Maximum Material Thickness",
  "Maximum Measurable Voltage",
  "Maximum Media Hardness",
  "Maximum Misalignment Angle",
  "Maximum Number of Circuits",
  "Maximum Number of Hoods",
  "Maximum Number of Padlocks",
  "Maximum Number of Respirators",
  "Maximum O.D Covering",
  "Maximum Operating Force",
  "Maximum Operating Temperature",
  "Maximum Output",
  "Maximum Output Current",
  "Maximum Outside Diameter",
  "Maximum Overall Length",
  "Maximum Overload Voltage",
  "Maximum Page Yield",
  "Maximum Permissible Power",
  "Maximum Pipe Size Compatibility",
  "Maximum Pipe Tap Capacity",
  "Maximum Piping Length",
  "Maximum Position Capacity",
  "Maximum Pull",
  "Maximum PV Rating",
  "Maximum Radius",
  "Maximum Reamer Diameter",
  "Maximum Recorder Output",
  "Maximum Reel Width",
  "Maximum Relief Pressure",
  "Maximum Resolution",
  "Maximum Rod Diameter",
  "Maximum Rod Outside Diameter",
  "Maximum Roll Diameter",
  "Maximum Room Size",
  "Maximum Run",
  "Maximum Run/Tap",
  "Maximum Safe Operating Pressure",
  "Maximum Sanding Depth",
  "Maximum Screw Adjustment",
  "Maximum Service Height",
  "Maximum SEU",
  "Maximum Shaft Fillet Radius",
  "Maximum Shank Diameter",
  "Maximum Shank Height",
  "Maximum Shank Width/Diameter",
  "Maximum Shell Reamer Diameter",
  "Maximum Slide Travel",
  "Maximum Spindle Inside Diameter",
  "Maximum Spread",
  "Maximum Static Pressure",
  "Maximum Steady Load",
  "Maximum Stroke",
  "Maximum Suction Lift",
  "Maximum Surface Finish",
  "Maximum Surge Current",
  "Maximum Synchronous Rate",
  "Maximum Talk Range",
  "Maximum Tap Capacity",
  "Maximum Tap Size",
  "Maximum Tap Size - Pipe (NPT)",
  "Maximum Temperature",
  "Maximum Tension",
  "Maximum Thickness",
  "Maximum Threads per Inch",
  "Maximum Throat Depth",
  "Maximum Tie Width",
  "Maximum Torque",
  "Maximum Torque Input",
  "Maximum Torque Output",
  "Maximum Trade Size",
  "Maximum Transformer Load",
  "Maximum Travel Distance",
  "Maximum Use Voltage",
  "Maximum User Codes",
  "Maximum Vacuum",
  "Maximum Velocity",
  "Maximum Viscosity",
  "Maximum Wall Thickness",
  "Maximum Wattage Rating",
  "Maximum Wheel Diameter",
  "Maximum Wheel Thickness",
  "Maximum Width of Cut",
  "Maximum Width of Dado",
  "Maximum Wire Diameter",
  "Maximum Wire Gage",
  "Maximum Wire Size",
  "Maximum Work Thickness",
  "Maximum Working Depth",
  "Maximum Working Pressure",
  "Maximum Working Space",
  "Maximum Working Voltage",
  "Maximum Workpiece Diameter",
  "Maximum Workpiece Thickness",
  "Maximum Workpiece Weight",
  "Maximum Workpiece Width",
  "Measurement Distance",
  "Measurement of Material Humidity",
  "Measurement Scale Included",
  "Measurement Type",
  "Measures",
  "Measuring Angle",
  "Measuring Force",
  "Measuring speed",
  "Measuring System",
  "Measuring Units",
  "Mechanical Durability",
  "Mechanical Seal Size",
  "Media Capacity",
  "Media Material",
  "Media Shape",
  "Media Temperature",
  "Media Type",
  "Megapixels",
  "Melting Range",
  "Melting Temperature",
  "Member Length",
  "Membrane Type",
  "Memory Recall",
  "Memory Resident",
  "Memory Size",
  "Memory Storage",
  "MERV",
  "Mesh Size",
  "Mesh Size Range",
  "Messenger Cable Diameter",
  "Metal Gauge Capacity",
  "Metal Parts Material",
  "Meter Size",
  "Metering Device",
  "Method of Manufacture",
  "Micron Grade",
  "Microphone",
  "Microwatts",
  "Midsole Material",
  "Milling Tip Material Grade",
  "Milling Tip Style",
  "Milling Tip Type",
  "Minimum Adjustable Size",
  "Minimum Application Temperature",
  "Minimum Arbor Diameter",
  "Minimum Arc Swing",
  "Minimum Back-Flex Radius",
  "Minimum Between Hooks",
  "Minimum Bore Diameter",
  "Minimum Bore Diameter - Largest Bar",
  "Minimum Bore Diameter - Smallest Bar",
  "Minimum Box Size",
  "Minimum Bundle Diameter",
  "Minimum Cable Diameter",
  "Minimum Cable/Hose Length",
  "Minimum Ceiling Height",
  "Minimum Centerline Height",
  "Minimum Code Size",
  "Minimum Crush Load",
  "Minimum Cutter Diameter",
  "Minimum Cutting Diameter",
  "Minimum Diameter",
  "Minimum Diameter Capacity",
  "Minimum Drill Blank Size",
  "Minimum Drill Diameter",
  "Minimum Drill Point Diameter",
  "Minimum Drill Size",
  "Minimum Duct Depth",
  "Minimum Embedment Depth",
  "Minimum End Space",
  "Minimum External Bend Radius",
  "Minimum Flagpole Height",
  "Minimum Floor Load",
  "Minimum Flow Rate",
  "Minimum Focus",
  "Minimum Forming Radius",
  "Minimum Grit",
  "Minimum Groove Width",
  "Minimum Hardness",
  "Minimum Head Diameter",
  "Minimum Height",
  "Minimum Hole Depth",
  "Minimum Hole Diameter",
  "Minimum Hose Size",
  "Minimum Included Angle",
  "Minimum Input Pulse Width",
  "Minimum Inside Diameter",
  "Minimum Internal Bend Radius",
  "Minimum Length",
  "Minimum Lifting Height",
  "Minimum Light Level",
  "Minimum Line Range",
  "Minimum Media Hardness",
  "Minimum Number of Clips",
  "Minimum Number of Cycles",
  "Minimum Operating Temperature",
  "Minimum Outside Diameter",
  "Minimum Overall Length",
  "Minimum Pipe Tap Capacity",
  "Minimum Planing Length",
  "Minimum Press Size",
  "Minimum Pulley Diameter",
  "Minimum Radius",
  "Minimum Radius Curve",
  "Minimum Reamer Diameter",
  "Minimum Relief Pressure",
  "Minimum Retracted Length",
  "Minimum Run/Tap",
  "Minimum Service Height",
  "Minimum Shank Diameter",
  "Minimum Shank Height",
  "Minimum Shank Width/Diameter",
  "Minimum Shell Reamer Diameter",
  "Minimum Side-Flex Radius",
  "Minimum Spindle Inside Diameter",
  "Minimum Sprocket Teeth",
  "Minimum Starting Temperature",
  "Minimum Surface Finish",
  "Minimum Tap Capacity",
  "Minimum Tap Size",
  "Minimum Tap Size - Pipe (NPT)",
  "Minimum Temperature",
  "Minimum Thickness",
  "Minimum Thread Length",
  "Minimum Threads per Inch",
  "Minimum Torque",
  "Minimum Trade Size",
  "Minimum Wheel Diameter",
  "Minimum Wire Size",
  "Minimum Workpiece Diameter",
  "Minimum Workpiece Thickness",
  "Mirror Dimensions",
  "Mirror Size",
  "Misalignment",
  "Miter Capacity",
  "Mix Ratio",
  "Mixer Diameter",
  "Mixing Ratio",
  "Mode",
  "Model Compatibility",
  "Model Number Compatibility",
  "Modes",
  "Modular (Yes/No)",
  "Modular Connection Size",
  "Modular Connection Thread Size",
  "Modular Head Style",
  "Modular Jack Blocks",
  "Modular System Size",
  "Modularity",
  "Modulation Frequencies",
  "Module Bank Density",
  "Module Capacity",
  "Module Data Width",
  "Module Interface",
  "Module Position",
  "Module Type",
  "Module Update Time",
  "Moisture",
  "Moisture Content",
  "Mold Bushing",
  "Mold Connection Type",
  "Mold Price Key",
  "Mold Size",
  "Mold Stop Blocks",
  "Molecular Weight",
  "Monitor",
  "Monitor Resolution",
  "Monitor Size",
  "Monitor Type",
  "Monitor Voltage",
  "Monitoring Function",
  "Motherboard Support",
  "Motion Sensing",
  "Motor Ambient Temperature Rating",
  "Motor Connection",
  "Motor Lead Size",
  "Motor Location",
  "Motor Rotation",
  "Motor Size",
  "Motor Speed",
  "Motor Timing",
  "Mount Compatibility",
  "Mount Size",
  "Mount Taper Size",
  "Mount Type",
  "Mounting Angle",
  "Mounting Application",
  "Mounting Bolt Diameter",
  "Mounting Bolt Hole Diameter",
  "Mounting Bolt Size",
  "Mounting Bolt Spacing",
  "Mounting Centers",
  "Mounting Depth",
  "Mounting Hardware Material",
  "Mounting Hole Center",
  "Mounting Hole Center to Center",
  "Mounting Hole Diameter",
  "Mounting Hole Quantity",
  "Mounting Hole Spacing",
  "Mounting Hole to Edge",
  "Mounting Location",
  "Mounting Pad Size",
  "Mounting Plate Thickness",
  "Mounting Pole Size",
  "Mounting Position",
  "Mounting Size",
  "Mounting Slot Size",
  "Mounting Slots",
  "Mounting Thread",
  "Mouth Type",
  "Movement Type",
  "Multi-Use Tool",
  "Multiple Display",
  "Nail Size",
  "Nap Size",
  "Neck Diameter",
  "Neck Size",
  "Neck Thread Size",
  "Neck Type",
  "Needle Diameter",
  "Needle Length",
  "NEMA Color",
  "NEMA Lug Spacing",
  "NEMA Size",
  "Nest Style",
  "Net Force Exerted",
  "Net Free Area",
  "Net IBR Rating",
  "Net Weight",
  "Network Compatibility",
  "Network Connection",
  "Network Interface",
  "Network Management and Protocols",
  "Network Service",
  "Network Technology",
  "Network Type",
  "Network Zoning",
  "Networking Specification",
  "Neutralizes",
  "Nipple Length",
  "Nipple Thread Size",
  "No of Infrared LEDs",
  "No of Labels per Card",
  "No-Load Speed",
  "Noise Figure Type",
  "Noise Level",
  "Noise Reduction",
  "Noise Reduction Rating",
  "Nominal Ball Diameter",
  "Nominal Bulb Size",
  "Nominal Cross Section Conductor",
  "Nominal Diameter",
  "Nominal Duct Diameter",
  "Nominal Force",
  "Nominal Input Impedance",
  "Nominal Pleat",
  "Nominal Rail Size",
  "Nominal Shaft Diameter",
  "Nominal Wall Size",
  "Noncontact RPM Range",
  "Normally Off Temperature",
  "Normally On Temperature",
  "Nose Angle",
  "Nose Diameter",
  "Nose Hole Thread Size",
  "Nose Material",
  "Nose Radius",
  "Nosepiece Size",
  "Noseplate Depth",
  "Noseplate Width",
  "Notch Angle",
  "Notch Spacing",
  "Notch Style",
  "Notch Type",
  "Notch Width",
  "Nozzle Diameter",
  "Nozzle Inside Diameter",
  "Nozzle Size",
  "Nozzle Spray Angle",
  "Nozzle Type",
  "Nozzles Included",
  "Number of Abrasive Sides",
  "Number of AC Outputs",
  "Number of Air Inlets",
  "Number of Analog Channels",
  "Number of Anchor Points",
  "Number of Antennas",
  "Number of Arms",
  "Number of Attachments",
  "Number of Axis",
  "Number of Bands",
  "Number of Barb",
  "Number of Bars",
  "Number of Basins",
  "Number of Batteries",
  "Number of Batteries Included",
  "Number of Beams",
  "Number of Belts",
  "Number of Bins",
  "Number of Bits",
  "Number of Bits per Cutter",
  "Number of Blades",
  "Number of Blades Included",
  "Number of Bolt Holes",
  "Number of Bolts",
  "Number of Bowls",
  "Number of Braces",
  "Number of Breakers",
  "Number of Built-In Shelves",
  "Number of Bulbs",
  "Number of Bumpers",
  "Number of Burners",
  "Number of Buttons",
  "Number of Cabinet Shelves",
  "Number of Cabinets",
  "Number of Cables",
  "Number of Camera Inputs",
  "Number of Candles",
  "Number of Capacitors",
  "Number of Cards",
  "Number of Cartridges Required",
  "Number of Casters",
  "Number of Chamfer Ring Inserts",
  "Number of Channels",
  "Number of Characters",
  "Number of Chemicals Dispensed",
  "Number of Chisels",
  "Number of Clamp Bolts",
  "Number of Cleaning Pads",
  "Number of Coils",
  "Number of Collars",
  "Number of Columns",
  "Number of Compartments",
  "Number of Components",
  "Number of Computers Controlled",
  "Number of Configuration Units",
  "Number of Connectors at End A",
  "Number of Connectors at End B",
  "Number of Controlled Axes",
  "Number of Controllers",
  "Number of Cores",
  "Number of Current Channels",
  "Number of Cutter Inserts",
  "Number of Cutting Edges",
  "Number of Cutting Knives",
  "Number of DC Powerpole Inputs",
  "Number of Decades",
  "Number of Devices",
  "Number of Divider Slots",
  "Number of Divider Slots Long",
  "Number of Divider Slots Small",
  "Number of Dividers",
  "Number of Door Shelves",
  "Number of Drawers",
  "Number of Drill Inserts",
  "Number of Drivers",
  "Number of Drums",
  "Number of Dryer Cycles",
  "Number of Edges",
  "Number of Elements",
  "Number of Embedded",
  "Number of Expansion Racks",
  "Number of Faucet Holes",
  "Number of Faucet Mount Holes",
  "Number of Faucets",
  "Number of Feeds",
  "Number of Fibers",
  "Number of Free Coils",
  "Number of Fuses",
  "Number of Geared Teeth",
  "Number of Grab Bars",
  "Number of Grates",
  "Number of Grooves",
  "Number of Handles",
  "Number of Hard Drives",
  "Number of Head Positions",
  "Number of Heat Settings",
  "Number of Hinges",
  "Number of Hole Sizes",
  "Number of Hooks",
  "Number of Hubs",
  "Number of Indexing Increments",
  "Number of Inlets",
  "Number of Input Circuits",
  "Number of Inserts",
  "Number of Inserts Used in Cartridge",
  "Number of Jaw Pieces",
  "Number of Jets",
  "Number of Key Hooks",
  "Number of Knives",
  "Number of Knockouts",
  "Number of Knurls Required",
  "Number of Labels",
  "Number of Latches",
  "Number of Layers",
  "Number of Leads",
  "Number of Leaves",
  "Number of Legs",
  "Number of Lenses",
  "Number of Level Vials",
  "Number of Leveling Screws",
  "Number of Lids",
  "Number of Lines",
  "Number of Link",
  "Number of Links",
  "Number of Magnets",
  "Number of Meter Sockets",
  "Number of Motors",
  "Number of Mounting Bolts",
  "Number of Mounting Holes",
  "Number of Mounting Points",
  "Number of Non-Pilot Inserts",
  "Number of Nozzles",
  "Number of Nuts",
  "Number of Openings",
  "Number of Pages",
  "Number of Panels",
  "Number of Pennants",
  "Number of Phases",
  "Number of Pieces",
  "Number of Pieces in Set",
  "Number of Pins",
  "Number of Pipes",
  "Number of Plates",
  "Number of Plumb Vials",
  "Number of Plys",
  "Number of Pockets",
  "Number of Points",
  "Number of Ports/Ways",
  "Number of Positions",
  "Number of Posts per Kit",
  "Number of Printed Sides",
  "Number of Probes",
  "Number of Prongs",
  "Number of Provisions",
  "Number of Pulling Speeds",
  "Number of Pulse Events",
  "Number of Pumps",
  "Number of Punches",
  "Number of Racks",
  "Number of Rail Holes",
  "Number of Relay Inputs",
  "Number of Relay Outputs",
  "Number of Relays",
  "Number of Removable Dividers",
  "Number of Resistance Channels",
  "Number of Ribs",
  "Number of Rings",
  "Number of Rod Eye Grooves",
  "Number of Rollers",
  "Number of Rolls",
  "Number of Rows",
  "Number of Rungs",
  "Number of Runs",
  "Number of Sampling Spans",
  "Number of Screws",
  "Number of Seals",
  "Number of Sections",
  "Number of Segments",
  "Number of Sensors",
  "Number of Settings",
  "Number of Sews",
  "Number of Sheds",
  "Number of Sheets",
  "Number of Sheets per Mat",
  "Number of Shelves/Bins",
  "Number of Shims Required",
  "Number of Sides",
  "Number of Sinks",
  "Number of Sizes",
  "Number of Spaces",
  "Number of Specimens",
  "Number of Speeds",
  "Number of Spindle Speeds",
  "Number of Spindles",
  "Number of Splices",
  "Number of Spray Functions",
  "Number of Spray Nozzles",
  "Number of Spray Type",
  "Number of Sprays",
  "Number of Stages",
  "Number of Starters",
  "Number of Starts",
  "Number of Stations",
  "Number of Stays",
  "Number of Steps",
  "Number of Strands",
  "Number of Straps",
  "Number of Stripes",
  "Number of Suspension Points",
  "Number of Switch Inputs",
  "Number of Switches",
  "Number of T-Slots",
  "Number of Tabs",
  "Number of Tangs",
  "Number of Tapped Holes",
  "Number of Tappings",
  "Number of Taps",
  "Number of Temp Settings",
  "Number of Temperature Channels",
  "Number of Temperature Positions",
  "Number of Temperature Setting",
  "Number of Test Unit",
  "Number of Tests",
  "Number of Threading Inserts",
  "Number of Threads",
  "Number of Timings",
  "Number of Tones",
  "Number of Tools",
  "Number of Towel Bars",
  "Number of Trolleys",
  "Number of Turns",
  "Number of Untapped Holes",
  "Number of USB Outputs",
  "Number of Users",
  "Number of Vacuum Holes",
  "Number of Vanes",
  "Number of Vials",
  "Number of Voltage Channels",
  "Number of Wall Panels",
  "Number of Wash Cycles",
  "Number of Ways",
  "Number of Wedges",
  "Number of Wheels",
  "Number of Wire Leads",
  "Number of Wires/Insulators",
  "Number of Zones",
  "Nut Finish",
  "Nut Height",
  "Nut Length",
  "Nut Load Capacity",
  "Nut Torque Rating",
  "Nut Type",
  "Object Size",
  "Observation Depth",
  "OD Clamping Range",
  "Offset",
  "Offset Angle",
  "Offset Size",
  "Ohms",
  "Oil Capacity",
  "Oil Charge Amount",
  "Oil Connection",
  "Oil Cup Capacity",
  "Oil Displacement Rating",
  "Oil Resistance",
  "Oil Type",
  "Open Diameter",
  "Open End Thickness",
  "Open End Width",
  "Open Length",
  "Open Pressure Rating",
  "Open Surface Area",
  "Open Width",
  "Opening Depth",
  "Opening Diameter",
  "Opening Height",
  "Opening Length",
  "Opening Mechanism",
  "Opening Required",
  "Opening Type",
  "Opening Width",
  "Operate Time",
  "Operating Direction",
  "Operating Pressure",
  "Operating Range",
  "Operating Speed",
  "Operating System",
  "Operating Time",
  "Operating Type",
  "Operating Velocity",
  "Operation",
  "Operation Range",
  "Operation Time",
  "Operational Limits",
  "Optical Connector",
  "Optical Power Range",
  "Optical Tube",
  "Optimum Range",
  "Orbit Type",
  "Orbits per Minute",
  "Orifice Size",
  "ORP Range",
  "Ounces per Stroke",
  "Outer Diameter",
  "Outer Diameter Range",
  "Outer Material",
  "Outer Tube Material",
  "Outlet Capacity",
  "Outlet Center Distance",
  "Outlet Configuration",
  "Outlet Connection",
  "Outlet Hose Connection Size",
  "Outlet Opening",
  "Outlet Pipe Size",
  "Outlet Port",
  "Outlet Pressure Range",
  "Outlet Rating",
  "Outlet Tap Size",
  "Outlet Valve Type",
  "Output Air Capacity",
  "Output Amperage",
  "Output Attenuation",
  "Output Capacity",
  "Output Circuits",
  "Output Conductor",
  "Output Current",
  "Output Drive Size",
  "Output Drive Type",
  "Output Frequency",
  "Output Function Type",
  "Output Impedance",
  "Output Jack",
  "Output Power",
  "Output Pressure",
  "Output Rate",
  "Output Relay Contacts",
  "Output Signal",
  "Output Size",
  "Output Speed",
  "Output Stability",
  "Output Torque",
  "Output Values",
  "Output Wattage",
  "Output Whips",
  "Outside Depth",
  "Outside Diameter of Pipe",
  "Outside Diameter Tolerance",
  "Outside Diameter Tubing",
  "Outside Dimensions",
  "Outside Edge Finish Angle",
  "Outside Eye Diameter",
  "Outside Flange Diameter",
  "Outside Height",
  "Outside Hub Size",
  "Outside Length",
  "Outside Modular Connection Size",
  "Outside Morse Taper Size",
  "Outside Shank Diameter",
  "Outside Square Size",
  "Outside Width",
  "Outsole Material",
  "Oven Output",
  "Overall Base Width",
  "Overall Diameter",
  "Overall Dimensions",
  "Overall Length Range",
  "Overall Length Tolerance",
  "Overall Length/Diameter",
  "Overall Reach",
  "Overall Thickness",
  "Overflow Height",
  "Overflow Yes/No",
  "Overhung Load",
  "Overload Capacity",
  "Oversize Pilot Model Number Compatibility",
  "Oxygen Concentration",
  "Package Material",
  "Package Size",
  "Package Type",
  "Packaging Type",
  "Packet Size",
  "Packing Spaces",
  "Pad Color",
  "Pad Diameter",
  "Pad Dimensions",
  "Pad Grip Method",
  "Pad Length",
  "Pad Material",
  "Pad Mounting Holes",
  "Pad Shape",
  "Pad Size",
  "Pad Thickness",
  "Pad Width",
  "Padlock Shackle Maximum Diameter",
  "Paint Type",
  "Pair Count",
  "Pair Number",
  "Pallet Size",
  "Palm Material",
  "Panel Capacity",
  "Panel Cutout Size",
  "Panel Dimensions",
  "Panel Hole Diameter",
  "Panel Thickness",
  "Panel Type",
  "Paper Color",
  "Paper Weight",
  "Parallelism",
  "Particle Size",
  "Particulate Removal Rate",
  "Pascal Range",
  "Pascal Resolution",
  "Pattern",
  "PC Interface",
  "Peak Amperage",
  "Peak Current",
  "Peak Energy",
  "Peak Gate Current",
  "Peak Gate Power",
  "Peak Hold",
  "Peak Load Current",
  "Peak Output Power",
  "Peak Repetitive Off-State Voltage",
  "Peak Repetitive Reverse Voltage",
  "Peak Reverse Gate Voltage",
  "Peak Surge On-Current",
  "Pedal Type",
  "People Served",
  "Per Position Loading",
  "Perforated",
  "Performance Temperature",
  "Period",
  "pH Accuracy",
  "Phase Error",
  "Phase Rotation",
  "Phase Type",
  "Phase Voltage Present Indicator",
  "Phillips Size",
  "Phone Entry Type",
  "Photocontrol Type",
  "Physical Dimensions",
  "Pigtail",
  "Pile Height",
  "Pillow Block Type",
  "Pilot Diameter",
  "Pilot Drill Diameter",
  "Pilot Drill Size",
  "Pilot Drill Style",
  "Pilot Duty Rating",
  "Pilot Hole Diameter",
  "Pilot Length",
  "Pilot Shank Diameter Compatibility",
  "Pilot Size",
  "Pilot Type",
  "Pin Height",
  "Pin Length",
  "Pin Size",
  "Pin Test",
  "Pinhole Diameter",
  "Pipe Capacity",
  "Pipe Center Size",
  "Pipe Diameter",
  "Pipe Material",
  "Pipe Schedule",
  "Pipe Size",
  "Pipe Thickness",
  "Pipe Thread Size",
  "Pipe Wall Thickness",
  "Pipe/Tube Size",
  "Pipe/Tube Thickness",
  "Piston Material",
  "Pitch Line to Back",
  "Pitch/Row Spacing",
  "Pivot Angle",
  "Pivoting Fence Angle",
  "Pixel Resolution",
  "Pixels",
  "Planer Size",
  "Plate Color",
  "Plate Height",
  "Plate Hole Size",
  "Plate Hole Spacing",
  "Plate Size",
  "Plate Style",
  "Plate Thickness",
  "Plate Thickness Compatibility",
  "Plate Type",
  "Plate Width",
  "Platform Depth",
  "Platform Height",
  "Platform Length",
  "Platform Material",
  "Platform Style",
  "Platform Type",
  "Platform Width",
  "Plating",
  "Plating Type",
  "Plenum Opening",
  "Plenum Size",
  "Pliers Type",
  "Plug Color",
  "Plug Configuration",
  "Plug Depth",
  "Plug Diameter",
  "Plug Material",
  "Plug Size",
  "Plunge Depth",
  "Plunger Diameter",
  "Plunger Length",
  "Plunger Material",
  "Plunger Projection",
  "Pocket Clip Included",
  "Point Diameter",
  "Point Length",
  "Point Shape",
  "Point Shape Code",
  "Point Size",
  "Point Style",
  "Polarity",
  "Polarization",
  "Polarizing Filter",
  "Pole",
  "Pole Circumference",
  "Pole Diameter",
  "Pole Distance",
  "Pole Height",
  "Pole Length",
  "Pole Material",
  "Pole Mounting Hole Size",
  "Pole Size",
  "Pole Spacing",
  "Pole Style",
  "Poles",
  "Pop Up Shape",
  "Port Adaptor",
  "Port Size",
  "Port Speed",
  "Port Standard Designation",
  "Porting Angle",
  "Position",
  "Post Base Size",
  "Post Color",
  "Post Diameter",
  "Post Finish",
  "Post Height",
  "Post Material",
  "Post Size",
  "Post/Joist Size",
  "Pour Point",
  "Powder Style",
  "Power Accuracy",
  "Power Cable Length",
  "Power Connector",
  "Power Frequency",
  "Power Frequency Withstand Voltage",
  "Power Level",
  "Power Method",
  "Power Port Slots",
  "Power Rating",
  "Power Type",
  "Powering Capacity",
  "Pre-Pressure Rating",
  "Precharge Rating",
  "Prepurge Time",
  "Prerequisite",
  "Pressure Angle",
  "Pressure Class",
  "Pressure Connection",
  "Pressure Drop",
  "Pressure Loss",
  "Pressure Port",
  "Pressure Range",
  "Pressure Required for Maximum Length of Cut",
  "Pressure Resolution",
  "Pressure Setting Range",
  "Pressure Switch Mounting",
  "Primary Ring Material",
  "Primary SDRAM Width",
  "Print Speed",
  "Print Type",
  "Printable Area",
  "Printer Compatibility",
  "Printhead",
  "Probe Connector",
  "Probe Depth",
  "Probe Diameter",
  "Probe Included",
  "Probe Length",
  "Probe Material",
  "Probe Ratio",
  "Probe Resistance",
  "Probe Size",
  "Probe Type",
  "Probes",
  "Process Connection",
  "Processor",
  "Processor Type",
  "Product Compatibility",
  "Product Number Compatibility",
  "Profile",
  "Profile Angle",
  "Profile Shape",
  "Program Capacity",
  "Programmable Burst Length",
  "Programmable LCD Control",
  "Programs per Week",
  "Projection (Yes/No)",
  "Projection From Edge",
  "Proof Test Pressure",
  "Propellant",
  "Propeller Diameter",
  "Propeller Material",
  "Propeller Size",
  "Propeller Speed",
  "Proportional Band",
  "Protected Height",
  "Protection degree",
  "Protection Rating",
  "Protrusion Size",
  "Publisher",
  "Pull Cable Length",
  "Pull Capacity",
  "Pull Chain Length",
  "Pull Force",
  "Pull to Lift Rated Load",
  "Pulling Capacity",
  "Pulling Force",
  "Pulling Speed",
  "Pullout",
  "Pulse Frequency",
  "Pump Capacity",
  "Pump Head Material",
  "Pump Inlet Size",
  "Pump Material",
  "Pump Outlet Size",
  "Pump Size",
  "Pump Type",
  "Punch Angle",
  "Punch Capacity",
  "Punch Diameter",
  "Punch Size",
  "Punch Type",
  "Punching Capacity in Mild Steel",
  "Punching Capacity in Stainless Steel",
  "Purified Water Resistivity",
  "Push Cable Diameter",
  "Push Rod Length",
  "Q Factor",
  "Quick Change (Yes/No)",
  "Quick Change System",
  "Quick Release Head",
  "Quill Diameter",
  "R Factor",
  "Rabbeting Capacity",
  "Rabbeting Depth",
  "Rack Groove Width",
  "Rack Space",
  "Rack Type",
  "Rack Unit",
  "Rack Units Size",
  "Radial Load Capacity",
  "Radial Rake Angle",
  "Radiated RF Power",
  "Radio Band Type",
  "Radio System Type",
  "Radius",
  "Rail Depth",
  "Rail Diameter",
  "Rail Length",
  "Rail Material",
  "Rail Shape",
  "Rail Size",
  "Rail Type",
  "Raised",
  "Raised Height",
  "RAM",
  "Ram Capacity",
  "Ram Force",
  "Ram Memory",
  "Ram Shape",
  "Ram Travel",
  "Range",
  "Range of Motion",
  "Rapid Traverse",
  "Ratchet Type",
  "Ratcheting",
  "Ratcheting Jaw",
  "Rate of Recovery",
  "Rated Head",
  "Rated Total Pressure",
  "Ratio",
  "Ratio Accuracy",
  "Ratio Analysis",
  "Reach",
  "Reaction System",
  "Reaction Time",
  "Read Range",
  "Reader Type",
  "Reading Distance",
  "Reading Width",
  "Reamer Diameter",
  "Reamer Size",
  "Reamer Size Measurement System",
  "Rear Wheel Diameter",
  "Rear Wheel Width",
  "Receiver Material",
  "Receiver to Transmitter Distance",
  "Reception Range",
  "Recessed Trim Type",
  "Recharge Time",
  "Rechargeable Upto",
  "Recommended Torque",
  "Recorder Voltage",
  "Recording Capacity",
  "Recording Modes",
  "Recording Time",
  "Recovered Insulation Diameter",
  "Recovered Wall Thickness",
  "Recovery Rate",
  "Recovery Time",
  "Reduced Pressure Adjustment Range",
  "Reel Capacity",
  "Reel Diameter",
  "Reel Material",
  "Reel Mounting",
  "Reel Mounting Type",
  "Reel Size",
  "Reel Type",
  "Reel Weight",
  "Reel Width",
  "Reeving",
  "Reflector Type",
  "Refresh Cycles",
  "Refrigerant",
  "Refrigerant Capacity",
  "Refrigerant Holding Capacity",
  "Refrigerant Pressure",
  "Refrigeration Dryer Performance",
  "Refrigerator Capacity",
  "Reinforced",
  "Reinforced Material",
  "Reinforcement Method",
  "Relay",
  "Relay Configuration",
  "Relay Output",
  "Relay Rating",
  "Relay Specification",
  "Relay Type",
  "Release Delay",
  "Release Time",
  "Release Version",
  "Relief Angle",
  "Relief Style",
  "Remaining Force",
  "Remote Handling",
  "Removability",
  "Removes",
  "Repeatability",
  "Repetitive Peak Off-State Voltage",
  "Repetitive Peak Reverse Voltage",
  "Replaceable Blade",
  "Replaceable Face",
  "Replaceable Jaws",
  "Replaceable Tip",
  "Replacement Blade",
  "Replaces",
  "Reponse Time",
  "Required Brush Speed",
  "Reserve Cranking Minutes",
  "Reservoir Capacity",
  "Reset Time",
  "Resettable",
  "Resin Type",
  "Resist Properties",
  "Resistance Accuracy",
  "Resistance Properties",
  "Resistance Range",
  "Resistant To",
  "Resistive Rating",
  "Resistivity",
  "Resistivity Accuracy",
  "Resistivity Range",
  "Resists",
  "Restores Pipe Threads",
  "Retainer Style",
  "Retainer Type",
  "Retractable Blade",
  "Retracted Length",
  "Retraction Force",
  "Retraction Type",
  "Retries",
  "Return Air Opening",
  "Return Port Size",
  "Reverse Current",
  "Reverse Leakage Current",
  "Reverse Recovery Time",
  "Reverse Voltage",
  "Reversible",
  "Reversing",
  "Revolutions per Minute",
  "Rewind Type",
  "RF Channels",
  "RF Communication Range",
  "RF Communications",
  "RF Input Power",
  "RF Sensitivity",
  "Rim Height",
  "Rim Style",
  "Ring Diameter",
  "Ring Material",
  "Ring Outside Diameter",
  "Ring Quantity",
  "Ring Size",
  "Ring Thickness",
  "Ring Tone",
  "Ring Type",
  "Ring Width",
  "Ripple Factor",
  "Rise",
  "Riser Height",
  "Riser Tube Diameter",
  "Rising Ratio of Off-Voltage",
  "Rivet Diameter",
  "Rivet Length",
  "Rivet Size",
  "RMS On-State Current",
  "Rockwell Hardness",
  "Rod Diameter",
  "Rod End Size",
  "Rod Length",
  "Rod Material",
  "Rod Size Tap",
  "Rod Thread",
  "Rod Thread Length",
  "Rod Thread Size",
  "Rod Type",
  "Rod/Track Length",
  "Roll Diameter",
  "Roll Length",
  "Roll Width",
  "Roller Diameter",
  "Roller Finish",
  "Roller Length",
  "Roller Material",
  "Roller Position",
  "Roller Size",
  "Roller Type",
  "Roller Width",
  "Rollers",
  "Rollers for Conveyor Section",
  "Rolling Capacity",
  "Roof Flange",
  "Root Diameter",
  "Rope Diameter",
  "Rope Length",
  "Rope Material",
  "Rope Size",
  "Rope Turn Back",
  "Rope Type",
  "Rotation",
  "Rotation Angle",
  "Rotation Type",
  "Rotational Direction",
  "Rotational Life",
  "Rotor Inertia",
  "Rough-In Size",
  "Roughness Parameter",
  "Router Style",
  "RPM",
  "Run Capacitor",
  "Run Size",
  "Run Time",
  "Run/Main Size",
  "Rung Diameter",
  "Rung Type",
  "Rung Width",
  "Running Bushing",
  "S/N Rating",
  "S/N Ratio",
  "Saddle Length",
  "Saddle Width",
  "SAE Number",
  "SAE/Metric",
  "Safe Air Pressure Rating",
  "Safe Pressure Rating",
  "Safe Temperature Rating",
  "Safe Working Load",
  "Safe Working Radius",
  "Safety Factor",
  "Safety Latch",
  "Safety Output Rating",
  "Safety Rating",
  "Safety Valve Size",
  "Salinity Accuracy",
  "Salinity Range",
  "Salt Storage Capacity",
  "Sample Rate",
  "Sample Time",
  "Sampling Length",
  "Sampling Method",
  "Sampling Mode",
  "Sampling Rate",
  "Sampling Time",
  "Scale Range",
  "Scale Type",
  "Scan Rate",
  "Scan Speed",
  "Scanner",
  "Scanning Width",
  "SCBA Backframe Material",
  "SCBA Harness Material",
  "Schedule Rating",
  "Screen Color",
  "Screen Material",
  "Screen Size",
  "Screw",
  "Screw Closure Size",
  "Screw Compatibility",
  "Screw Diameter",
  "Screw Hole Size",
  "Screw Length",
  "Screw Load Capacity",
  "Screw Material",
  "Screw Pitch",
  "Screw RPM",
  "Screw Style",
  "Screw Torque Rating",
  "Screw Type",
  "Screw/Stud Size",
  "Screwdriver Size Range",
  "Screwdriver Types Included",
  "Seal Head Outside Diameter",
  "Seal Material",
  "Seal Type",
  "Sealing Element Inside Diameter",
  "Sealing Element Outside Diameter",
  "Seam Style",
  "Seat Back Height",
  "Seat Back Width",
  "Seat Depth",
  "Seat Dimensions",
  "Seat Height",
  "Seat Shape",
  "Seat Size",
  "Seat Surface Temperature Rating",
  "Seat Thickness",
  "Seat Type",
  "Seat Width",
  "Seated Amperage",
  "Seated Wattage",
  "Seating",
  "Section Size",
  "Security Type",
  "SEER Rating",
  "Segment Height",
  "Sensing Angle",
  "Sensing Bulb Size",
  "Sensing Element",
  "Sensing Medium",
  "Sensitivity Level",
  "Sensor Length",
  "Sensor Model",
  "Sensor Range",
  "Sensor Size",
  "Sensor Technology",
  "Serial Connector",
  "Serial Port",
  "Series Compatibility",
  "Service Covered",
  "Service Factor",
  "Service Life",
  "Service Test Pressure",
  "Set Contents",
  "Set Pressure",
  "Set Pressure Range",
  "Set Range",
  "Set Type",
  "Setpoint",
  "Setscrew",
  "Setter Height",
  "Setting Range",
  "Setting Time",
  "SEU Opening",
  "Sew Color",
  "Sew Material",
  "Maximum Shackle Diameter",
  "Shackle Height",
  "Shackle Material",
  "Shackle Width",
  "Shaft Connection",
  "Shaft Contact Pressure",
  "Shaft Diameter",
  "Shaft Diameter Range",
  "Shaft Height",
  "Shaft Material",
  "Shaft Rotation",
  "Shaft Seal Material",
  "Shaft Shape",
  "Shaft Speed",
  "Shaft Style",
  "Shaft Type",
  "Shank Connection Size",
  "Shank Diameter Compatibility",
  "Shank Diameter Workpiece Side",
  "Shank Height",
  "Shank Length",
  "Shank Material",
  "Shank Opening",
  "Shank Shape",
  "Shank Taper Designation",
  "Shank Taper Size",
  "Shank Thread Length",
  "Shank Thread Size",
  "Shank Width",
  "Shape Code",
  "Shape of Conductor",
  "Shape Type",
  "Sharpening Material",
  "Shear Strength",
  "Shearing Blade Length",
  "Sheath Diameter",
  "Sheath Included",
  "Sheath Material",
  "Sheave Inside Diameter",
  "Sheave Material",
  "Sheave Outside Diameter",
  "Sheave Width",
  "Shed Diameter",
  "Sheet Capacity",
  "Sheet Material",
  "Sheet Size",
  "Shelf Adjustments",
  "Shell Finish",
  "Shell Size",
  "Shell Style",
  "Shell Type",
  "Shield Type",
  "Shielding Type",
  "Shim Style",
  "Shock",
  "Shock Resistance",
  "Shoe Group Size",
  "Shoe Material",
  "Short Circuit Current Rating",
  "Short Circuit Duration",
  "Short Run Air Consumption",
  "Shoulder Diameter",
  "Shoulder Height",
  "Shoulder Length",
  "Shoulder Strap Buckle",
  "Shower Arm Length",
  "Shower Flow Rate",
  "Shower Head Diameter",
  "Shower Head Patterns",
  "Shower Head Size",
  "Shower Size",
  "Shrink Temperature",
  "Shrink Time",
  "Shrinking Dia. Range First Coil",
  "Shunt Wound Voltage",
  "Shut-Off Type",
  "Shutoff Head",
  "Side A Opening",
  "Side Angle",
  "Side B Opening",
  "Side Cutter",
  "Side Cutting Edge Angle",
  "Side Rail Style",
  "Side Return Size",
  "Side Set Degree",
  "Side Space",
  "Side Spray(Y/N)",
  "Sight Diameter",
  "Sight Length",
  "Sign Capacity",
  "Sign Height",
  "Sign Material",
  "Sign Sheeting",
  "Sign Width",
  "Signal Bandwidth",
  "Signal Input",
  "Signal to Noise Ratio",
  "Signal Type",
  "Signaling System",
  "Single or Double End",
  "Single or Double Ended",
  "Sink Gauge",
  "Sink Shape",
  "Sink Type",
  "Size Code",
  "Size Compatibility",
  "Size Number Designation",
  "Size Range",
  "Size Type",
  "Skatewheel Material",
  "Skew Angle",
  "Slack Tube Length",
  "Sleep Mode",
  "Sleeve Diameter",
  "Sleeve Length",
  "Sleeve Size",
  "Sleeve Thread",
  "Sleeves Furnished",
  "Slide",
  "Slide Height",
  "Slide Length",
  "Slider Size",
  "Sling Rating",
  "Slip Roll Diameter",
  "Slip Roll Length",
  "Slot Capacity",
  "Slot Type",
  "Slot Width",
  "Slots",
  "Slotted",
  "Slotting Cutter Outside Diameter",
  "Small End Diameter",
  "Snap Opening",
  "Snap Size",
  "Snip Type",
  "Snug Type",
  "Soaking Depth",
  "Soaking Time",
  "Socket Amps",
  "Socket Cover Style",
  "Socket End Width",
  "Socket Hex Size",
  "Socket Jaws",
  "Socket Length",
  "Socket Material",
  "Socket Shape",
  "Socket Size",
  "Socket Type",
  "Sockets",
  "Sockets Included",
  "Software Requirements",
  "Software Title",
  "Soil Type",
  "Solar Panel Power Output",
  "Solar Power Technology",
  "Solderability",
  "Sole Type",
  "Solid Rod Capacity",
  "Solids Content",
  "Solids Handling",
  "Solubility",
  "Solution Properties",
  "Solution Type",
  "Sones",
  "Sound Pressure",
  "Sound Type",
  "Space Factor",
  "Spacer Size",
  "Spacer Through Hole Size",
  "Spacing",
  "Span",
  "Speaker Output",
  "Speaker Position",
  "Speaker Size",
  "Speaker Type",
  "Special Features",
  "Specialty Point Size",
  "Specific Gravity",
  "Spectral Bandwidth",
  "Spectral Response",
  "Speed at No Load",
  "Speed Control",
  "Speed of Feeding",
  "Speed Rating",
  "Speed Setting",
  "Sphere Diameter",
  "Spigot Size",
  "Spill Capacity",
  "Spindle Clearance",
  "Spindle Diameter",
  "Spindle Nose",
  "Spindle Nut Size",
  "Spindle Size",
  "Spindle Speed",
  "Spindle Taper",
  "Spindle Thread Size",
  "Spindle to Column Distance",
  "Spindle to Table Distance",
  "Spindle Travel",
  "Spindle Type",
  "Spindle Vertical Travel",
  "Spindle/Drill Capacity",
  "Spiral Angle",
  "Spiral Type",
  "Splice Length",
  "Splice Size",
  "Splice Type",
  "Splices Capacity",
  "Spline Outside Diameter",
  "Split Length",
  "Split Tolerance",
  "Spool Width",
  "Spot Face Diameter",
  "Spout Finish",
  "Spout Height",
  "Spout Hose Thread Type",
  "Spout Length",
  "Spout Reach",
  "Spout Type",
  "Spray Angle",
  "Spray Distance Horizontal",
  "Spray Distance Vertical",
  "Spray Head Finish",
  "Spray Head Material",
  "Spray Hose Length",
  "Spray Hose Material",
  "Spray Pattern",
  "Spray Temperature",
  "Spray Type",
  "Spraying Action",
  "Spread",
  "Spread Width",
  "Spreading Range",
  "Spring Constant",
  "Spring Deflection",
  "Spring Length",
  "Spring Material",
  "Spring Range",
  "Spring Rate",
  "Spring Return",
  "Spring Tension",
  "Spring Type",
  "Sprinkler",
  "Sprinkler Size",
  "Spud Coupling",
  "Spud Location",
  "Spud Size",
  "Spud Type",
  "Square Length",
  "Square Opening Required",
  "Square Post",
  "Square Size",
  "Square Tube Size",
  "Squareness",
  "Stack Temperature Rating",
  "Stages",
  "Stages Cool",
  "Stages Heat",
  "Stair Angle",
  "Stair Width",
  "Standard Pilot Model Number Compatibility",
  "Standard Speed",
  "Standoff",
  "Standout",
  "Staple Capacity",
  "Staple Length",
  "Staple Size",
  "Staple Type",
  "Stapling Position",
  "Star Diameter",
  "Start Capacitor",
  "Starter Tip/Nut Type",
  "State/Province",
  "Static Dissipative",
  "Static Pressure",
  "Static Pressure Water Level",
  "Steam Outlet Size",
  "Steam Production Rating",
  "Steel Wool Grade",
  "Steel Wool Grade Number",
  "Stem & Pin Material",
  "Stem Density",
  "Stem Diameter",
  "Stem Hole Size",
  "Stem Length",
  "Stem Material",
  "Stem Size",
  "Stem Type",
  "Step Angle",
  "Step Depth",
  "Step Height",
  "Step Length",
  "Step Size",
  "Step Type",
  "Step Width",
  "Steps per Revolution",
  "Stock Shape",
  "Stock Size",
  "Stock Thickness",
  "Storage Capacity",
  "Storage Conditions",
  "Storage Controller",
  "Storage Dimensions",
  "Storage Specification",
  "Storage Temperature",
  "Storage Unit Type",
  "Storage/Operating Temperature",
  "Strand Diameter",
  "Strand Size",
  "Strand Type",
  "Strands",
  "Strap Finish/Color",
  "Strap Inner Diameter",
  "Strap Length",
  "Strap Material",
  "Strap Size",
  "Strap Thickness",
  "Strap Width",
  "Strapping Thickness",
  "Strapping Width",
  "Stream",
  "Strength",
  "Strength Rating",
  "Stretch Limit",
  "Strike Distance",
  "Strike Height",
  "Strike Length",
  "Strike Plate Size",
  "Strip Length",
  "Stripe Color",
  "Stripe Height",
  "Stripe Material",
  "Stroke",
  "Stroke Frequency",
  "Stroke Length",
  "Stroke Type",
  "Strokes per Minute",
  "Strut Size",
  "Strut Spring Diameter",
  "Stud Diameter",
  "Stud Hole Spacing",
  "Stud Holes",
  "Stud Material",
  "Stud Size",
  "Stud Spacing",
  "Stud Type",
  "Subject",
  "Suction Area",
  "Suction Lift",
  "Suction Line Size",
  "Suction Tube Diameter",
  "Suitable Conveyor Chain Number",
  "Suitable Fluids",
  "Suitable Media",
  "Suitable Sprocket Diameter",
  "Supply Air Opening",
  "Supply Opening Depth",
  "Supply Opening Size",
  "Supply Opening Width",
  "Supply Size",
  "Support Formats",
  "Support Material",
  "Support Rollers",
  "Support Type",
  "Surface Application",
  "Surface Area per Plate",
  "Surface Finish",
  "Surface Material",
  "Surface Pattern",
  "Surface Size",
  "Surface Type",
  "Surge Protection Voltage",
  "Suspenders Type",
  "Suspension Type",
  "Swab Length",
  "Sweep Face",
  "Sweep Range",
  "Sweep Rate",
  "Sweep Type",
  "Sweeping Width",
  "Swept Area",
  "Swing",
  "Swing Over Bed",
  "Swing Over Cross Slide",
  "Switch Action",
  "Switch Amperage Rating",
  "Switch Color",
  "Switch Contact",
  "Switch Off Time",
  "Switch On Time",
  "Switch Output",
  "Switch Poles",
  "Switch Positions",
  "Switch Rating",
  "Switch Style",
  "Switch Type",
  "Switch Wires",
  "Switches",
  "Switching",
  "Switching Capacity",
  "Switching Time",
  "Switching Type",
  "Swivel Angle",
  "Swivel Material",
  "Swivel Pad",
  "Swivel Size",
  "Swivel Type",
  "System Compatibility",
  "System of Measurement",
  "System Requirements",
  "System Size",
  "System Size Machine Side",
  "System Size Workpiece Side",
  "System Switching",
  "System Type",
  "T-Slot Length",
  "T-Slot Size",
  "T-Slot Width",
  "Tab Color",
  "Tab Size",
  "Table Dimensions",
  "Table Height",
  "Table Length",
  "Table Load Capacity",
  "Table Size",
  "Table Tilt",
  "Table Travel",
  "Table Travel to Center of Bit",
  "Table Width",
  "Tailstock Spindle Travel",
  "Tailstock Taper Size",
  "Take Up",
  "Talk Channel Output",
  "Tall or Short",
  "Tandem",
  "Tang",
  "Tang Size",
  "Tang Style",
  "Tangent Crank",
  "Tank Capacity",
  "Tank Depth",
  "Tank Dimensions",
  "Tank Height",
  "Tank Inlet Size",
  "Tank Length",
  "Tank Material",
  "Tank Mounting",
  "Tank Orifice Size",
  "Tank Pressure",
  "Tank Size",
  "Tank Type",
  "Tank Volume",
  "Tank Width",
  "Tank Width/Diameter",
  "Tap Capacity",
  "Tap Chamfer Type",
  "Tap Code",
  "Tap Depth",
  "Tap Drill Size",
  "Tap Finish/Coating",
  "Tap Material",
  "Tap Quantity",
  "Tap Shank Diameter",
  "Tap Size",
  "Tap Square Size",
  "Tap Stud",
  "Tap Thread Size",
  "Tap Type",
  "Tape Brake",
  "Tape Color",
  "Tape Core Diameter",
  "Tape Delivery Length",
  "Tape Diameter",
  "Tape Leg Length",
  "Tape Length",
  "Tape Material",
  "Tape Profile",
  "Tape Size",
  "Tape Type",
  "Tape Width",
  "Taper",
  "Taper Length",
  "Taper Pin Size/Number",
  "Taper Size",
  "Taper Size Designation",
  "Tapered End Diameter",
  "Tapped Hole Size",
  "Tapping",
  "Tapping Diameter",
  "Target Emissivity",
  "Target Material",
  "Target Size",
  "Teeth Type",
  "Temp Lock",
  "Temp Setting",
  "Temper",
  "Temperature Accuracy",
  "Temperature at Tip",
  "Temperature Coefficient",
  "Temperature Lock",
  "Temperature Resistance",
  "Temperature Rise",
  "Temperature Setting",
  "Temperature Settings per Day",
  "Temperature Trip Range",
  "Temple Style",
  "Tension Adjustment",
  "Tension Adjustment Method",
  "Tension Capacity",
  "Terminal Connection Type",
  "Terminal Designations",
  "Terminal Material",
  "Terminal Size",
  "Termination Type",
  "Test Frequency",
  "Testing",
  "Testing Parameter",
  "Tether Capable",
  "Tether Ready",
  "Thermal Characteristics",
  "Thermal Conductivity",
  "Thermal Cutout",
  "Thermal Protection",
  "Thermal Resistance",
  "Thermal Time Constant",
  "Thermocouple Type",
  "Thermostat Type",
  "Thickness Tolerance",
  "Thimble Diameter",
  "Thimble Type",
  "Thread Angle",
  "Thread Diameter",
  "Thread Lead",
  "Thread Length",
  "Thread Limit",
  "Thread Locking Element",
  "Thread Profile",
  "Thread Restorer Compatibility",
  "Thread Standard",
  "Thread Starts",
  "Threaded Mount Size",
  "Threaded Shank Size",
  "Threading Stand Type",
  "Threads",
  "Threads per Inch",
  "Three Hole Head Adjustment",
  "Threshold Level",
  "Throat",
  "Throat Depth",
  "Throat Size",
  "Throttle Style",
  "Throttling Range",
  "Through Coolant (Yes/No)",
  "Thrust Direction",
  "Tie Entry",
  "Tilling Depth",
  "Tilting Angle",
  "Time Base Frequency",
  "Time Base Stability",
  "Time Cycle",
  "Time Format",
  "Time Period",
  "Timed Entry Allowance",
  "Timer Range",
  "Timer Rating",
  "Tine Material",
  "Tip Angle",
  "Tip Cap Shape",
  "Tip Diameter",
  "Tip Finish",
  "Tip Length",
  "Tip Material",
  "Tip Material Grade",
  "Tip Shape",
  "Tip Size",
  "Tip Texture",
  "Tip Thickness",
  "Tip Width",
  "Tips",
  "Tips Included",
  "TIR",
  "Tire Material",
  "Tire Rim Diameter",
  "Tire Size",
  "Tissue Count",
  "Tissue Size",
  "Toe Type",
  "Tolerance",
  "Tone Frequency",
  "Tone Injection",
  "Tone Type",
  "Tongue Size",
  "Tongue Type",
  "Tongue Width",
  "Tonnage Capacity",
  "Tool Angle",
  "Tool Bit Compatibility",
  "Tool Bit Holder Compatibility",
  "Tool Block Style",
  "Tool Capacity",
  "Tool Compatibility",
  "Tool Construction",
  "Tool Holder Capacity",
  "Tool Holder Compatibility",
  "Tool Holder Insert Dimension",
  "Tool Holder Material",
  "Tool Holder Size",
  "Tool Holder Style",
  "Tool Holder Style Compatibility",
  "Tool Length",
  "Tool Light",
  "Tool Material",
  "Tool Post Height",
  "Tool Post Holder Number",
  "Tool Size",
  "Tool Storage Included",
  "Tool Storage Type",
  "Tool Type",
  "Tool/Kit",
  "Toolholder Application",
  "Toolholder Material",
  "Toolholder Type",
  "Tools",
  "Tools Included",
  "Tooth Angle",
  "Tooth Configuration",
  "Tooth Form",
  "Tooth Material",
  "Tooth Pitch",
  "Tooth Profile",
  "Tooth Set",
  "Tooth Style",
  "Top Color",
  "Top Cover Size",
  "Top Diameter",
  "Top Dimensions",
  "Top Edge Type",
  "Top Frame Length",
  "Top Frame Thickness",
  "Top Frame Width",
  "Top Material",
  "Top Step Height",
  "Top Style",
  "Top Terminal",
  "Top Thickness",
  "Top to Bottom Rung",
  "Top to Inlet Center",
  "Top Width",
  "Torque Range",
  "Torque Ratio",
  "Torx Size",
  "Torx Size Range",
  "Total Capacity",
  "Total Harmonic Distortion",
  "Total Number of Bins",
  "Total Number of Circuits per Box",
  "Total Projection",
  "Total Taps",
  "Total Uncertainty",
  "Total Zoom",
  "TPI",
  "TPI or Pitch",
  "Traces Wires in Conduit",
  "Traces Wires in Floors",
  "Traces Wires in Non-Metallic Conduit",
  "Traces Wires in Walls",
  "Track Length",
  "Track Type",
  "Trade Type",
  "Traffic Time",
  "Transfer Time",
  "Transformer Rating",
  "Transformer Setting",
  "Transmissible Thrust",
  "Transmissible Torque",
  "Transmission Channel",
  "Transmission Mode",
  "Transmission Range",
  "Transmission Speed",
  "Transmitting Velocity",
  "Transverse Direction",
  "Trap Primer Threads",
  "Trapway Size",
  "Travel Adjustment",
  "Travel Length",
  "Tray Flange Type",
  "Tray Length",
  "Tray Size",
  "Trays Capacity",
  "Tread",
  "Tread Pattern",
  "Tread Type",
  "Trench Depth",
  "Triac Output",
  "Trigger Type",
  "Trim Color",
  "Trim Kit",
  "Trim Length",
  "Trim Size",
  "Trim Type",
  "Trip Delay",
  "Trip Range",
  "Tripod Screw Size",
  "Trolley Load Capacity",
  "Trough Angle",
  "Trough Type",
  "Trun On Time",
  "Tub Depth",
  "Tub Length",
  "Tub Width",
  "Tube Color",
  "Tube Dash Number",
  "Tube Diameter",
  "Tube Length",
  "Tube Material",
  "Tube Outside Diameter",
  "Tube Profile",
  "Tube Quantity",
  "Tube Size",
  "Tube Type",
  "Tubing Capacity",
  "Tubing Compatibility",
  "Tubing Material",
  "Tuning Range",
  "Turbidity Range",
  "Turbidity Resolution",
  "Turning Radius",
  "Turns Ratio Range",
  "Twine Size",
  "Twisted Pair Cable Standard",
  "Two Hole Head Adjustment",
  "Type of Power",
  "Type of Wire",
  "UART Rate",
  "Ultimate Strength",
  "Ultimate Stress",
  "Ultraviolet Output",
  "Uniform Distributed Load",
  "Unit Size",
  "Unwrapping Terminal Hole Depth",
  "Unwrapping Terminal Hole Diameter",
  "Update Version",
  "Upper & Midsole Material",
  "Upper Die Height",
  "Upper Die Width",
  "Upper Hole Diameter",
  "Upper Material",
  "Upper Oven Capacity",
  "Upright Post Diameter",
  "Upright Post Length",
  "Usable Hours per Week",
  "Usable Length",
  "Usable Opening Depth",
  "Usage Barring Method",
  "Use Arbor",
  "Use Divider Number",
  "Use Lid Number",
  "User Interface",
  "UV Protection",
  "V-Groove Angle",
  "Vacuum Inlet",
  "Vacuum Rating",
  "Valve Configuration",
  "Valve Design",
  "Valve Flow Rate",
  "Valve Inlet Connection",
  "Valve Material",
  "Valve Outlet Connection",
  "Valve Series",
  "Valve Size",
  "Valve Type",
  "Valve/Cartridge Type",
  "Vapor Transmission",
  "Variable Diameter Reamer Type",
  "Variable Feed Rate",
  "Variable Torque Speed Range",
  "Varistor Voltage",
  "Velocity",
  "Vent Hole Size",
  "Vent Size",
  "Vent Tapping",
  "Vent Type",
  "Vent/Duct Size",
  "Vertical Bus Rating",
  "Vertical Clearance",
  "Vertical Graduations",
  "Vertical Hitch Capacity",
  "Vertical Range",
  "Vertical Rise Height",
  "Vial Material",
  "Vial Positions",
  "Vial Type",
  "Vibration Absorption",
  "Vibration Level",
  "Vibration Rating",
  "Vibration Resistance",
  "Vibrations/Minute",
  "Video Bandwidth",
  "Video Compatibility",
  "Video Format",
  "Video I/O",
  "Video I/O Type",
  "Video Inputs",
  "Video Output Mode",
  "Video Outputs",
  "Video Specification",
  "Video System Input",
  "Viewing Aperture Area",
  "Viewing Aperture Size",
  "Viewing Distance",
  "Viscosity Index",
  "Viscosity Rating",
  "Visibility",
  "Visible Discharge Voltage",
  "Visor Color",
  "Visor Height",
  "Visor Material",
  "Visor Thickness",
  "Visor Width",
  "Visual Alert",
  "Visual/Audible Indicators",
  "Voltage Accuracy",
  "Voltage Drop",
  "Voltage Gain",
  "Voltage Measurement",
  "Voltage Measuring Range",
  "Voltage Protection",
  "Voltage Range",
  "Voltage Regulation Percentage",
  "Voltage Sensing Range",
  "Voltage Standing Wave Ratio (VSWR)",
  "Volume",
  "Volume Capacity",
  "Volume Control",
  "Waist Size",
  "Walk Through Size",
  "Walkway",
  "Wall Opening Size",
  "Wall Pattern",
  "Wall Thickness",
  "Wall Thickness Recovered",
  "Wall Type",
  "Wallplate Type",
  "Wand Material",
  "Wand Size",
  "Warm on Temperature",
  "Warm-Up Time",
  "Washer Capacity",
  "Washer Diameter",
  "Washer Size",
  "Waste Curl",
  "Water Capacity",
  "Water Connection",
  "Water Feed Rate",
  "Water Inlet Size",
  "Water Reservoir",
  "Water Surface Area",
  "Water Temperature",
  "Water Type",
  "Water/Steam Content",
  "Waterproof",
  "Waterproof Grade",
  "Wattage Consumption",
  "Wattage Density",
  "Wavelength",
  "Wavelength Range",
  "Wax Thickness",
  "Weave Pattern",
  "Web Grade",
  "Webbing Material",
  "Weight",
  "Weight Capacity",
  "Weight per Foot",
  "Weld Type",
  "Welded Material",
  "Welding Position",
  "Welding Process",
  "Well Depth",
  "Wet/Dry Operation",
  "Wetted Material",
  "Wetted Materials",
  "Wheel Angle",
  "Wheel Bearings",
  "Wheel Capacity",
  "Wheel Circumference",
  "Wheel Color",
  "Wheel Density",
  "Wheel Diameter",
  "Wheel Hardness",
  "Wheel Inside Diameter",
  "Wheel Load Rating",
  "Wheel Material",
  "Wheel Outside Diameter",
  "Wheel Shape",
  "Wheel Size",
  "Wheel Speed",
  "Wheel Thickness",
  "Wheel Tread Diameter",
  "Wheel Type",
  "Wheel Type Number",
  "Wheel Width",
  "Wheel/Disc Diameter",
  "Wheel/Disc Material",
  "Wheels Center-Center",
  "Wheels per Foot",
  "Width Across Fork",
  "Width Between Eyes",
  "Width Between Fork",
  "Width Offset",
  "Width/Diameter",
  "Winch Cable Diameter",
  "Winch Cable Length",
  "Winch Cable Material",
  "Winch Cable Tensile Strength",
  "Winch Power Type",
  "Wind Direction",
  "Wind Load Capacity",
  "Winder Case (Yes/No)",
  "Window Height",
  "Window Size",
  "Window Thickness",
  "Window Width",
  "Wing Span",
  "Wing Spread",
  "Wire Capacity",
  "Wire Count",
  "Wire Diameter",
  "Wire Diameter Range",
  "Wire Gauge",
  "Wire Groove Width",
  "Wire Grooves",
  "Wire Hole Size",
  "Wire Length",
  "Wire Lug Size",
  "Wire Material",
  "Wire Range",
  "Wire Rope Diameter",
  "Wire Shape",
  "Wire Spacing",
  "Wire Strip Length",
  "Wire Stripper",
  "Wire Stripper Included",
  "Wire Stripping Range",
  "Wire/Contact Size",
  "Wireless Range",
  "Wireway Size",
  "Wiring Angle",
  "Wiring Code",
  "With Slide Bar Yes/No",
  "Withstand Pull Force",
  "Withstand Rating",
  "Work Life",
  "Work Surface Depth",
  "Work Surface Material",
  "Work Surface Type",
  "Work Surface Width",
  "Work Thickness",
  "Workers per System",
  "Working Angle",
  "Working Depth",
  "Working Distance",
  "Working Height",
  "Working Length",
  "Working Load",
  "Working Pressure",
  "Working Range",
  "Working Space",
  "Working Temperature",
  "Working Time",
  "Working Torque",
  "Working Width",
  "Workpiece Capacity",
  "Workpiece Diameter",
  "Workpiece Diameter Capacity",
  "Workpiece Height",
  "Workpiece Length Capacity",
  "Workpiece Material",
  "Wrapping Terminal Hole Depth",
  "Wrapping Terminal Hole Diameter",
  "Wrench Opening",
  "Wrench Opening Type",
  "Wrench Square",
  "Wrench Type",
  "Writable Memory",
  "Writing Area",
  "X Axis Length",
  "Year Published",
  "Yoke Size",
  "Z Axis Length",
  "Zoom Type",
]

In [ ]:
# subset
doc1_elements:list=[
  "Type",
  "Series",
  "Container Type",
  "Container Size",
  "Net Weight",
  "Base Type",
  "Composition",
  "Form",
  "Color",
  "Odor/Scent",
  "Temperature Range",
  "Specific Gravity",
  "Flash Point",
  "pH Range",
  "VOC Content",
  "Viscosity",
  "Pour Point",
  "Flammability Rating",
  "Grade",
  "Inside Diameter",
  "Outside Diameter",
  "Width",
  "Thickness",
  "Cross Section",
  "Cross Section Shape",
  "Dash Number",
  "Durometer",
  "Hardness",
  "Temperature Rating",
  "Suitable For Use With",
  "Material",
  "Domestic/Import",
  "Backing",
  "Length",
  "Finish",
  "Tensile Strength",
  "Cell Type",
  "Density",
  "Stretch Limit",
  "Material Type",
  "Liner Material",
  "Shape",
  "Adhesive Material",
  "Backing Material",
  "Maximum Operating Temperature",
  "Minimum Operating Temperature",
  "Specifications Met",
  "End Style",
  "Wall Thickness",
  "Pressure Class",
  "Schedule Rating",
  "Working Pressure",
  "Method of Manufacture",
  "Yield Strength",
  "Nominal Size",
  "Number of Bolt Holes",
  "Bolt Hole Size",
  "Class",
  "Pressure Rating",
  "Industry Standards",
  "Diameter",
  "Outer Diameter",
  "Inner Diameter",
  "Fitting/Connector Type",
  "Hex Size",
  "Dimensions",
  "Bolt Circle Diameter",
  "Connection Type",
  "Hub Diameter",
  "Length Through Hub",
  "Bore Size",
  "Depth",
  "Height",
  "MERV",
  "Flow Rate",
  "Filter Rating",
  "Frame Type",
  "Efficiency",
  "Initial Resistance",
  "Final Resistance",
  "Media Material",
  "Power Source",
  "Sound Level",
  "Mounting Type",
  "Amperage Rating",
  "Wattage",
  "Life Hours",
  "Bulb Shape",
  "Microwatts",
  "Ultraviolet Output",
  "Coil Length",
  "Resistant To",
  "Frame Material",
  "Fabric Weight",
  "Connection Method",
  "Collection",
  "Bowl Size",
  "Drain Opening Size",
  "Number of Faucet Holes",
  "Faucet Hole Drilling",
  "Faucet Hole Spacing",
  "Grate Type",
  "Trap Primer Threads",
  "Number of Basins",
  "Overall Dimensions",
  "Number of Legs",
  "Leg Material",
  "Finish/Color",
  "Capacity",
  "Overall Length",
  "Overall Width",
  "Overall Height",
  "Specifications",
  "Size",
  "Voltage Rating",
  "Frequency Rating",
  "Chilling Capacity",
  "Gallons per Hour",
  "Number of Stations",
  "Installation Type",
  "Measurement System",
  "Diameter - Thread Size",
  "Head Type",
  "Drive Type",
  "Drive Size",
  "Head Size",
  "Thread Type",
  "Thread Length",
  "Point Type",
  "Point Size",
  "Thread Size",
  "Inside Diameter Tolerance",
  "Outside Diameter Tolerance",
  "Overall Length Tolerance",
  "Hex Width",
  "Hex Width Tolerance",
  "Material Grade",
  "Port Type",
  "Media",
  "Actuator Type",
  "Inlet to Outlet Length",
  "Top to Inlet Center",
  "Body Material",
  "Trim Material",
  "Softgoods Material",
  "Hand Wheel Diameter",
  "Low Lead Compliance",
  "Orifice Size",
  "Ball Material",
  "Disc Material",
  "Stem & Pin Material",
  "Length Under Head",
  "Material Specification",
  "Minimum Thread Length",
  "Neck Type",
  "Neck Size",
  "Shank Type",
  "Shank Size",
  "#N/A",
  "Chemical Composition",
  "Welding Position",
  "Impact Energy",
  "Applicable Materials",
  "Bore Diameter",
  "Flange Diameter",
  "Flange Width",
  "Groove/Plug Type",
  "Load Capacity",
  "Maximum Pressure",
  "Maximum Velocity",
  "Maximum PV Rating",
  "Bearing Material",
  "Plug Type",
  "Shaft Diameter",
  "Diameter Tolerance",
  "Hardness Depth",
  "Resin Type",
  "Vibration Absorption",
  "Chemical Type",
  "Drying Time",
  "Work Life",
  "Setting Time",
  "Curing Time",
  "Mixing Ratio",
  "Adheres to",
  "Wire Diameter",
  "Mesh Size",
  "Mesh Size Range",
  "Weave Pattern",
  "Opening Size",
  "Head Diameter",
  "Shoulder Diameter",
  "Shoulder Length",
  "Contactor Size",
  "Number of Poles",
  "Number of Contacts",
  "Contact Resistance",
  "Maximum Load Current",
  "Tip Type",
  "Insulation Type",
  "Maximum Contact Block",
  "Operating Cycles",
  "Operating Temperature",
  "Environmental Conditions",
  "Brush Size",
  "Wire Length",
  "Number of Pieces",
]

In [ ]:
doc2_elements:list=[
  "Cross Section - Fractional",
  "Cross Section Shape",
  "Dash Size",
  "Inner Diameter - Actual",
  "Inner Diameter - Fractional",
  "Outer Diameter - Actual",
  "Outer Diameter - Fractional",
  "Length (ft.)",
  "Taxonomy Level 5",
  "# of Bolt Holes",
  "Bolt Hole Diameter",
  "For Flange ANSI Class",
  "For Pipe Size",
  "Pressure Class",
  "Filler Material",
  "Outer Ring Material",
  "Sealing Element OD",
  "Sealing Element Thickness",
  "Inner Ring ID",
  "Shelf-life in days",
  "Vacuum Pressure Rating",
  "Compressibility ",
  "Maximum Pressure",
  "Recovery",
  "Tensile Strength ",
  "For Rotary Applications",
  "Reinforcement Material ",
  "Fits Edge Thickness",
  "For Hole ID",
  "Grommet Height",
  "Grommet ID",
  "Grommet OD",
  "For Sealing",
  "Spring Material",
  "Thick",
  "O-Ring Material",
  "Base Oil",
  "Lubricant Type",
  "Thickener",
  "Bend Radius",
  "Curing Method",
  "Operating Pressure",
  "Reinforcement",
  "Appearance",
  "Connection (A)",
  "Connection (B)",
  "For Flange OD",
  "For Tube OD",
  "Interior Smoothness,Ra",
  "Tube Connection Type",
  "For Flange OD (Reducing End)",
  "Connection (C)",
  "Connection (D)",
  "Pipe Size",
  "Hose Barb Size (For Tube ID)",
  "Ball Material",
  "Body Length",
  "Body Material",
  "Connection Type",
  "Handle Material",
  "Handle Type",
  "Seat Material",
  "Tube Size",
  "Valve Type",
  "Flow Coefficient (Cv)",
  "Gasket Material",
  "Glass Length",
  "Glass Material",
  "Clean Room",
  "For Tube Size",
  "With Screen",
  "Screen Material",
  "Gripping Ring Material",
  "Maximum Vacuum",
  "Pipe Thread Material",
  "Pipe Thread Sealant Material",
  "Pipe Thread Type",
  "Ring Release Color",
  "Ring Release Material",
  "Working Pressure",
  "Body Color",
  "Flared Angle",
  "Tube Thread Size",
  "Tube Thread Type",
  "Flange OD",
  "Flange OD (Reducer Side)",
  "Socket Connection Type",
  "Ring Type",
  "Hose Type",
  "For Plug Tip Diameter",
  "Latch Material",
  "Mounting Option",
  "Plug/Socket",
  "Plug/Socket Size",
  "Ring Material",
  "Compression End Size (For Tube OD)",
  "Plug Tip Diameter",
  "Coupler/Adapter",
  "Coupler/Adapter Size",
  "Plug OD",
  "Pressure Rating @ 72F",
  "Socket ID",
  "For Hose Coupling Size",
  "Connection Style",
  "Flanged Connection Surface",
  "For Bolt Diameter",
  "Number of Bolt Holes",
  "Pipe Fittings",
  "Pipe Nipples & Pipe",
  "Backing Type",
  "Cell Type",
  "Compression Deflection @ 25%",
  "Firmness",
  "Texture",
  "Adhesive Temperature Range",
  "Compression Set",
  "Durometer",
  "Elongation",
  "Tear Strength",
  "Thickness Tolerance",
  "Fabric Reinforcement",
  "Military Specs Met",
  "Coefficient of Friction",
  "Friction Rating",
  "Impact Strength",
  "Impact Strength Rating",
  "Length Tolerence",
  "Tensile Strength Rating",
  "Thickness Tolerence",
  "Water Absorption",
  "Water Absorption Rating",
  "Width Tolerance",
  "Abrasion Resistance",
  "Compression Deflection for 10%",
  "Felt Type",
  "Vibration Absorption",
  "Wool Content",
  "Compressibility",
  "Electrical Resistivity",
  "Fabrication",
  "Flex Strength",
  "Wall Thickness",
  "Cross Section - Actual",
  "Weave Style",
  "Plug Construction",
  "Pre-Applied Thread Sealant",
  "Schedule",
  "Thread",
  "Max Steam Pressure",
  "Pieces",
  "Port Type",
  "Stem Material",
  "Facing Material",
  "ABEC Rating",
  "Dynamic Radial Load Capacity",
  "Inch/Metric",
  "Maximum Speed",
  "Static Radial Load Capacity",
  "Style",
  "Temperature Range ",
  "Trade Number",
  "Width Tolerance ",
  "Length ",
  "Profile",
  "Vacuum Rating",
  "Electrical Properties",
  "Grain Size",
  "Maximum Temperature",
  "Maximum Vacuum Temperature",
  "Application",
  "Filter Rating",
  "Mask Attatchment Type",
  "Mask Size",
  "Total Units",
  "Edge Profile",
  "Height",
  "Installation Type",
  "Units per Set",
  "Self-Supported Divider Leg Height",
  "Self-Supported Divider Leg Slot Height",
  "Self-Supported Divider Leg Slot Width",
  "Self-Supported Divider Leg Thickness",
  "Self-Supported Divider Leg Width",
  "Self-Supported Divider Slot Edge Offset",
  "Self-Supported Divider Slot Height",
  "Self-Supported Divider Slot Width",
  "Edge Radius",
  "Divider Opening Height",
  "Divider Opening Location",
  "Divider Opening Offset Width",
  "Divider Opening Radius",
  "Divider Opening Width",
  "Divider Opening Offset Height",
  "# of Dividers",
  "Compartment Dimensions",
  "Cross Divider Length ",
  "Description",
  "for # of People",
  "Height ",
  "Rubber Bumper Included",
  "Spine Length ",
  "Table Dimension ",
  "Film Surface Finish",
  "Depth",
  "Taoxnomy Level 2",
  "Taoxnomy Level 4",
  "Inside Height",
  "Inside Width",
  "Outside Height",
  "Outside Height Tolerance",
  "Outside Width",
  "Outside Width Tolerance",
  "Bracket Recommendations",
  "Desk Type",
  "General Usage",
  "Number of Panels",
  "Panel Front Width",
  "Panel Height",
  "Panel Left Side Width",
  "Panel Right Side Width",
  "Panel Thickness",
  "Virco Model Compatability",
  "1st Slots for Headrest Mount Bottom Offset",
  "1st Slots for Headrest Mount Width Offset",
  "Car Type",
  "Console Film Height",
  "Console Film Thickness",
  "Console Film Width",
  "Distance between Slots for Headrest Mount (Height)",
  "Distance between Slots for Headrest Mount (Width)",
  "Divider Edge Profile",
  "Divider Edge Radius",
  "Divider Height",
  "Divider Thickness",
  "Divider Width",
  "Slots for Headrest Mount # of Rows",
  "Slots for Headrest Mount # of Velcro Strips",
  "Slots for Headrest Mount Height",
  "Slots for Headrest Mount Velcro Type",
  "Slots for Headrest Mount Width",
  "Slots per Row for Headrest Mount",
  "Base Type",
  "Edge Surface Height",
  "For Divider Thickness",
  "For Edge Thickness",
  "Mounting Surface",
  "Mounting Type",
  "Slot Type",
  "Adhesive Strength",
  "Tasonomy Level 1",
  "Tasonomy Level 3",
  "Installation Location",
  "Leg Base Width",
  "Leg Height",
  "Leg Panel Location",
  "Leg Quantity Included",
  "Panel Long Side Width",
  "Panel Short Side Width",
  "Rubber Bumper Quantity",
  "Rubber Bumper Size",
  "Surface Resistivity",
  "Air Purification Method",
  "Airflow",
  "Airflow at Speed",
  "Airflow Range",
  "CADR",
  "CADR Range",
  "Compatable Filter",
  "Cord Length",
  "Coverage",
  "Coverage Range",
  "Filter Composition",
  "Frequency",
  "HEPA Filter Grade",
  "Maximum Sound Level",
  "Mobility",
  "Mounting Location",
  "Number of Speeds",
  "Particle Efficiency",
  "Power",
  "Power Source",
  "Sound Level at Speed Setting",
  "Compatible Model",
  "Replacement Part Type",
  "Clarity",
  "Density",
  "For Light Type",
  "Hardness Rating",
  "Haze",
  "Light Transmission",
  "Pattern",
  "Resistance Properties",
  "Thickness",
  "Width",
  "Features",
  "Flammability Specifications",
  "Tasonomy Level 4",
  "Panel Front Length",
  "Panel Side Length",
  "Tasonomy Level 2",
  "Panel Length",
  "Panel Width",
  "Assembly",
  "Faucet Delivery",
  "Flow Rate",
  "Indoor/Outdoor",
  "Installation",
  "Shipping Method",
  "Warm Water",
  "Water Drainage",
  "Water Supply",
  "Durability Rating",
  "Operating Size",
  "Accessory Type",
  "Filter",
  "Refrigeration",
  "Retrofits Models",
  "Filter Capacity",
  "Filter Specs",
  "Drive Size",
  "Drive Style",
  "Finish",
  "Head Diameter",
  "Head Height",
  "Head Subtype",
  "Head Type",
  "Length",
  "Material",
  "Screw Subtype",
  "Screw Type",
  "System of Measurement",
  "Thread Direction",
  "Thread Fit",
  "Thread Size",
  "Thread Spacing",
  "Thread Type",
  "Threading",
  "Min Thread Length",
  "Thread Pitch",
  "Countersink Angle",
  "For Screw Size",
  "Hardness",
  "ID",
  "ID Tolerance",
  "Length Tolerance",
  "OD",
  "OD Tolerance",
  "Shape",
  "Type",
  "Female Thread Fit",
  "Female Thread Length",
  "Female Thread Pitch",
  "Female Thread Size",
  "Hex Size",
  "Hex Size Tolerance",
  "Male Thread Fit",
  "Male Thread Length",
  "Male Thread Pitch",
  "Male Thread Size",
  "Comparable To",
  "Maximum Pressure for Fluids",
  "Maximum Pressure for Gases",
  "Retainer Hardness",
  "Retainer Material",
  "Retainer Thickness",
  "Retainer Thickness Tolerance",
  "Seal Color",
  "Seal Hardness",
  "Seal Material",
  "Seal Thickness",
  "Seal Thickness Tolerance",
  "Sealing Washer Type",
  "Temperature Range",
  "Disc Material",
  "Flow Pattern",
  "Flange Connection Style",
  "Search Terms",
  "Bolt Subtype",
  "Bolt Type",
  "Head Subytype",
  "Head Width",
  "Seal Type",
  "Resistant To Oils",
  "Resistant To Water",
  "Resistant To Weather",
  "Resistant To Chemicals",
  "Resistant To High Temperature (400°F +)",
  "For Joining",
  "Weight",
  "Backing Material",
  "Coefficient of Friction - Dry",
  "Coefficient of Friction - With Oil/Grease",
  "Housing Bore",
  "Installed Bearing I.D",
  "Liner Material",
  "Material Dynamic Load Capacity",
  "Material Static Load Capacity ",
  "Maximum PV - Dry ",
  "Maximum PV - Oiled ",
  "Maximum Sliding Speed - Dry ",
  "Maximum Sliding Speed - With Oil/Grease ",
  "Shaft Diameter",
  "Thermal Conductivity",
  "Flange Diameter",
  "Diameter",
  "Diameter Tolerance",
  "Edge Type",
  "End Type",
  "For Motion Type",
  "Heat Treatment",
  "Straightness Tolerance",
  "Surface Smoothness",
  "Yield Strength",
  "Micron Rating",
  "Search Terms and Keywords for Website",
  "Chemical Base",
  "Color",
  "Cure Time",
  "Cure Type",
  "Model",
  "Operating Temperature",
  "Package Type",
  "Peel Strength",
  "Product Form",
  "Shear Strength",
  "Shore Hardness",
  "Specific Gravity",
  "Specifications Met",
  "Tensile Strength",
  "Working Time",
  "Size",
  "Melt Flow Rate",
  "Shrinkage",
  "For Use With",
  "LDA",
  "Construction",
  "Mesh Size",
  "Open Area",
  "Opening Size",
  "Wire Diameter",
  "Joist Size",
  "Screw Holes",
  "Screws Included",
  "Weight Capacity",
  "Head Width Tolerance",
  "Body Width",
  "Body Width Tolerance",
  "Head Height Tolerance",
  "Neck Type",
  "Flange Thickness",
  "Flange Width",
  "Lag Thread Length",
  "Machine Thread Length",
  "Center Space",
  "Head Diameter Tolerance",
  "Point Type",
  "Thread Subtype",
  "Tapping/Drilling Screw Type",
  "Handle Length",
  "Handle Width",
  "OEM Brand ",
  "OEM Device Number",
  "OEM Part Number",
  "OEM Series / Type",
  "OEM/Aftermarket",
  "Part Type",
  "Pole Set",
  "Amperage",
  "Configuration ",
  "Voltage",
  "Brush Length ",
  "Brush Material ",
  "Brush Thickness",
  "Brush Width ",
  "Grade",
  "OEM Brand",
  "Shunt Length",
  "Shunt Position",
  "Terminal Style",
  "Intermittent Swivel",
  "Tube Stem Diameter",
]

In [ ]:
# Fake Doc1
doc1_elements:list=[
  "As part of our construction project, we will install solar panels on the rooftop to harness renewable energy and reduce our carbon footprint. This initiative aligns with our commitment to sustainability and will contribute significantly to our energy independence.",
  "Construction of a new office building with sustainable materials and energy-efficient systems to minimize environmental impact.",
  "Construction of a parking structure for employees.",
  "Construction of bicycle storage facilities to encourage eco-friendly transportation choices.",
  "Construction of new office building with sustainable materials.",
  "Construction of parking structure for employee convenience.",
  "Creation of outdoor recreational spaces to promote employee well-being and encourage physical activity.",
  "Design and construction of pedestrian-friendly pathways to enhance accessibility and safety.",
  "Design and implementation of a green roof to enhance biodiversity and mitigate urban heat island effects.",
  "Design and implementation of green roof for biodiversity.",
  "Design and installation of green walls to improve air quality and provide natural insulation.",
  "Implementation of energy-efficient HVAC systems to improve indoor air quality and reduce operational costs.",
  "Implementation of rainwater harvesting systems for water reuse and sustainability.",
  "Implementation of waste management solutions to reduce landfill waste and promote recycling efforts.",
  "Installation of electric vehicle charging stations to support sustainable commuting options.",
  "Installation of smart lighting solutions for energy savings and improved lighting quality.",
  "Installation of solar panels on the rooftop to harness renewable energy and reduce carbon footprint.",
  "Integration of digital security systems for enhanced building and data protection.",
  "Integration of fire safety systems to ensure building protection and occupant safety.",
  "Landscaping with native plants to promote environmental sustainability and enhance aesthetic appeal.",
  "Our scope of work involves renovating the existing electrical systems to meet current safety standards and enhance operational efficiency. We will implement advanced technologies to optimize energy consumption while ensuring reliability and compliance with regulatory requirements.",
  "Renovation of existing electrical systems to meet current safety standards and optimize energy consumption.",
  "The construction of a new parking structure for employees is essential to alleviate current parking shortages and improve accessibility. Our design focuses on maximizing space efficiency and incorporating eco-friendly materials to enhance the overall environmental impact of the project.",
  "Upgrade of building insulation to improve energy efficiency and maintain indoor comfort.",
  "Upgrade of elevators to meet accessibility standards and enhance user experience.",
  "Upgrade of plumbing infrastructure to enhance water conservation and ensure sustainable usage.",
  "We are tasked with designing and implementing a green roof atop the new office building to enhance biodiversity, improve air quality, and mitigate urban heat island effects. Our approach integrates innovative planting techniques and sustainable irrigation systems.",
  "We propose to construct a state-of-the-art office building equipped with sustainable materials and advanced energy-efficient systems to minimize environmental impact and enhance employee productivity. The project includes innovative design elements such as a green roof and solar panels integrated into the building's structure.",
]

In [ ]:
# Fake Doc2
doc2_elements:list=[
  "Coverage for building insulation upgrades, protecting against energy loss and maintaining thermal efficiency.",
  "Coverage for elevator upgrades and installations, protecting against mechanical failures and ensuring safety compliance.",
  "Coverage for employee parking structure construction, protecting against accidents and liability claims during construction phases.",
  "Coverage for green wall design and installation, protecting against maintenance issues and structural damage.",
  "Coverage for liability during the construction of office buildings, protecting against property damage and third-party claims.",
  "Coverage for pedestrian-friendly pathway projects, protecting against accidents and ensuring safe access.",
  "Coverage for smart lighting solutions, protecting against electrical failures and ensuring continuous operation.",
  "Insurance for digital security system integration, covering data breaches and system failures impacting security measures.",
  "Insurance for electric vehicle charging stations, covering equipment damage and liability related to charging operations.",
  "Insurance for HVAC system installations, covering system failures and performance issues impacting operational efficiency.",
  "Insurance for landscaping projects with native plants, covering damage and loss due to natural disasters and accidents.",
  "Insurance for rainwater harvesting systems, covering equipment failures and water quality issues.",
  "Insurance for solar panel installation projects, covering equipment damage and performance issues due to adverse conditions.",
  "Insuring green roofs against damage from severe weather conditions, vandalism, and structural issues.",
  "Our insurance package includes coverage specifically tailored for the solar panel installation project, offering protection against equipment damage, performance issues, and financial losses due to adverse weather conditions or operational failures.",
  "Our insurance plan includes coverage for the green roof of the new office building, protecting against damage from severe weather conditions, vandalism, and structural issues. We emphasize proactive risk management to safeguard your sustainable investments.",
  "Our insurance policy provides comprehensive coverage for all liability risks during the construction phases of the new office building project. We ensure protection against property damage, third-party injuries, and unforeseen events to safeguard your investment.",
  "Policy for bicycle storage facilities construction, providing coverage for theft, damage, and liability claims.",
  "Policy for construction of outdoor recreational spaces, providing coverage for construction-related risks and liability.",
  "Policy for fire safety system integration, providing coverage for fire incidents and compliance-related liabilities.",
  "Policy for plumbing infrastructure upgrades, providing coverage for leaks, failures, and water damage.",
  "Policy for renovations and upgrades of electrical systems, providing coverage for equipment failures and business interruption.",
  "Policy for waste management solutions, providing coverage for environmental liabilities and waste disposal incidents.",
  "We offer specialized insurance solutions for the renovation of electrical systems, providing coverage for equipment failures, electrical fires, and business interruption. Our policies are designed to minimize financial risks and ensure business continuity.",
  "We provide comprehensive insurance coverage for the construction of employee parking structures, protecting against structural damage, accidents during construction, and liability claims. Our tailored solutions ensure peace of mind and financial security.",
]

##### Vector Embedding using Traditional Methods

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=None,
  doc2_elements_embedding=None,
  threshold_=0.7,
  output_folder='TF_IDF'
  ).main()

wrote the final output to local:  Sun Jun 23 14:28:38 2024


##### Vector Embedding using Word2Vec Models

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=text_to_vector.get_word_embedding(texts=doc1_elements,model_name=word_embedding_model),
  doc2_elements_embedding=text_to_vector.get_word_embedding(texts=doc2_elements,model_name=word_embedding_model),
  threshold_=0.6,
  output_folder='Word2Vec'
  ).main()

wrote the final output to local:  Sun Jun 23 14:28:57 2024


##### Vector Embedding using Laser Embedding

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=text_to_vector.get_laser_embeddings(texts=doc1_elements,model_name=laser_embeddings),
  doc2_elements_embedding=text_to_vector.get_laser_embeddings(texts=doc2_elements,model_name=laser_embeddings),
  threshold_=0.6,
  output_folder='Laser'
  ).main()

wrote the final output to local:  Sun Jun 23 14:29:10 2024


##### Vector Embedding using spaCy Model

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=text_to_vector.get_spacy_embedding(texts=doc1_elements,model_name=nlp_spacy_model),
  doc2_elements_embedding=text_to_vector.get_spacy_embedding(texts=doc2_elements,model_name=nlp_spacy_model),
  threshold_=0.6,
  output_folder='spaCy'
  ).main()

wrote the final output to local:  Sun Jun 23 14:29:21 2024


##### Vector Embedding using Sentence Transformer Model

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=text_to_vector.get_sentence_transformers_embedding(texts=doc1_elements,model_name=sentence_transformer_model),
  doc2_elements_embedding=text_to_vector.get_sentence_transformers_embedding(texts=doc2_elements,model_name=sentence_transformer_model),
  threshold_=0.6,
  output_folder='Sentence_Transformer'
  ).main()

wrote the final output to local:  Sun Jun 23 14:29:24 2024


##### Vector Embedding using Pre-Trained Model

In [ ]:
DocMapper(
  doc1_elements_list=doc1_elements,
  doc2_elements_list=doc2_elements,
  doc1_elements_embedding=text_to_vector.get_pre_trained_models_embedding(texts=doc1_elements,model_name=hf_model,model_tokenizer=hf_tokenizer,custom_max_length=256),
  doc2_elements_embedding=text_to_vector.get_pre_trained_models_embedding(texts=doc2_elements,model_name=hf_model,model_tokenizer=hf_tokenizer,custom_max_length=256),
  threshold_=0.6,
  output_folder='BERT'
  ).main()

wrote the final output to local:  Sun Jun 23 14:30:03 2024


### Cleaning RAM

In [ ]:
custom_ram_cleanup_func()
del custom_ram_cleanup_func